# Chen's Cosmetics Problem
Make sure you've installed `Plotly`!

In [1]:
import time

import pandas as pd
import plotly as py
import pulp
# py.offline.init_notebook_mode()

# Initial Setup

In [2]:
# To see the results of the run:
#      Go to the directory where you put this file and you'll see two output files
#      One output file will be the summary and one will have details
#      And, tabs will open if you decide to map the input and output

scenario_name = 'scenario_2'
# Change the service distance (in km) you'd like to select.
high_service_dist = 600
# Change the average service distance of all customers you'd like to select. 
# This could come from objective value of second model.
avg_service_dist = 1000
# Maximum distance (in km) a customer can be served from a warehouse.
maximum_dist = 5000
# Quantity of demand to be serviced within the service distance. 
# This could come from objective value of first model.
high_service_demand = 13164538  # start with default of 500100100.
# Change the number of warehouses you would like to select.  This number should be an integer
number_of_whs = 3
# Control drawing of the input and output maps
input_map = True
output_map = True

# Load Input Data

In [3]:
#  This is the input data. In a normal project, you would load this from a file.
#  We are making it simple by packaging it here
def get_data():
    # warehouses: {id:(name, city, state, zipcode, lat, long)}
    warehouses = {15: ('Changchun', 'Changchun', 'China', 43.87, 125.35),
                  17: ('Changsha', 'Changsha', 'China', 28.2, 112.97),
                  23: ('Chengdu', 'Chengdu', 'China', 30.67, 104.07),
                  25: ('Chongqing', 'Chongqing', 'China', 29.57, 106.58),
                  28: ('Dalian', 'Dalian', 'China', 38.92, 121.65),
                  40: ('Guangzhou', 'Guangzhou', 'China', 23.12, 113.25),
                  47: ('Hangzhou', 'Hangzhou', 'China', 30.25, 120.17),
                  48: ('Harbin', 'Harbin', 'China', 45.75, 126.65),
                  68: ('Jilin', 'Jilin', 'China', 43.85, 126.55),
                  69: ('Jinan', 'Jinan', 'China', 36.67, 117.0),
                  77: ('Jiulong', 'Jiulong', 'China', 22.32, 114.17),
                  101: ('Nanchang', 'Nanchang', 'China', 28.68, 115.88),
                  102: ('Nanjing', 'Nanjing', 'China', 32.05, 118.78),
                  109: ('Peking', 'Peking', 'China', 39.93, 116.4),
                  114: ('Qingdao', 'Qingdao', 'China', 36.07, 120.32),
                  122: ('Shanghai', 'Shanghai', 'China', 31.23, 121.47),
                  130: ('Shenyang', 'Shenyang', 'China', 41.8, 123.45),
                  133: ('Shijiazhuang', 'Shijiazhuang', 'China', 38.05, 114.48),
                  143: ('Taiyuan', 'Taiyuan', 'China', 37.87, 112.55),
                  147: ('Tianjin', 'Tianjin', 'China', 39.13, 117.2),
                  151: ('Urumqi', 'Urumqi', 'China', 43.8, 87.58),
                  154: ('Wuhan', 'Wuhan', 'China', 30.58, 114.27),
                  159: ('Xian', 'Xian', 'China', 34.27, 108.9),
                  169: ('Xinyang', 'Xinyang', 'China', 32.13, 114.07),
                  191: ('Zhengzhou', 'Zhengzhou', 'China', 34.75, 113.67)}

    # customers: {id:(name, city, country, lat, long)}
    customers = {1: ('Aksu', 'Aksu', 'China', 41.15, 80.25),
                 2: ('Anqing', 'Anqing', 'China', 30.5, 117.05),
                 3: ('Anshan', 'Anshan', 'China', 41.12, 122.95),
                 4: ('Anshun', 'Anshun', 'China', 26.25, 105.93),
                 5: ('Anyang', 'Anyang', 'China', 36.08, 114.35),
                 6: ('Aomen', 'Aomen', 'China', 22.27, 113.56),
                 7: ('Baicheng', 'Baicheng', 'China', 45.62, 122.82),
                 8: ('Baoding', 'Baoding', 'China', 38.87, 115.48),
                 9: ('Baoji', 'Baoji', 'China', 34.38, 107.15),
                 10: ('Baotou', 'Baotou', 'China', 40.6, 110.05),
                 11: ('Beihai', 'Beihai', 'China', 21.48, 109.1),
                 12: ('Bengbu', 'Bengbu', 'China', 32.95, 117.33),
                 13: ('Benxi', 'Benxi', 'China', 41.33, 123.75),
                 14: ('Cangzhou', 'Cangzhou', 'China', 38.32, 116.87),
                 15: ('Changchun', 'Changchun', 'China', 43.87, 125.35),
                 16: ('Changde', 'Changde', 'China', 29.03, 111.68),
                 17: ('Changsha', 'Changsha', 'China', 28.2, 112.97),
                 18: ('Changzhi', 'Changzhi', 'China', 35.22, 111.75),
                 19: ('Changzhou', 'Changzhou', 'China', 31.78, 119.97),
                 20: ('Chaoyang', 'Chaoyang', 'China', 41.55, 120.42),
                 21: ('Chaozhou', 'Chaozhou', 'China', 23.67, 116.64),
                 22: ('Chengde', 'Chengde', 'China', 40.96, 117.93),
                 23: ('Chengdu', 'Chengdu', 'China', 30.67, 104.07),
                 24: ('Chifeng', 'Chifeng', 'China', 42.27, 118.95),
                 25: ('Chongqing', 'Chongqing', 'China', 29.57, 106.58),
                 26: ('Chuzhou', 'Chuzhou', 'China', 32.3, 118.3),
                 27: ('Daipo', 'Daipo', 'China', 22.45, 114.16),
                 28: ('Dalian', 'Dalian', 'China', 38.92, 121.65),
                 29: ('Dandong', 'Dandong', 'China', 40.13, 124.4),
                 30: ('Datong', 'Datong', 'China', 40.08, 113.3),
                 31: ('Dezhou', 'Dezhou', 'China', 37.45, 116.3),
                 32: ('Dongchang', 'Dongchang', 'China', 41.73, 125.93),
                 33: ('Dongguan', 'Dongguan', 'China', 23.05, 113.73),
                 34: ('Dongying', 'Dongying', 'China', 37.5, 118.52),
                 35: ('Foshan', 'Foshan', 'China', 23.03, 113.12),
                 36: ('Fushun', 'Fushun', 'China', 41.87, 123.88),
                 37: ('Fuxin', 'Fuxin', 'China', 42.01, 121.65),
                 38: ('Fuzhou', 'Fuzhou', 'China', 26.08, 119.3),
                 39: ('Ganzhou', 'Ganzhou', 'China', 25.95, 114.93),
                 40: ('Guangzhou', 'Guangzhou', 'China', 23.12, 113.25),
                 41: ('Guilin', 'Guilin', 'China', 25.28, 110.28),
                 42: ('Guiyang', 'Guiyang', 'China', 26.58, 106.72),
                 43: ('Haibowan', 'Haibowan', 'China', 39.66, 106.8),
                 44: ('Haikou', 'Haikou', 'China', 20.05, 110.32),
                 45: ('Handan', 'Handan', 'China', 36.58, 114.48),
                 46: ('Hangu', 'Hangu', 'China', 39.23, 117.78),
                 47: ('Hangzhou', 'Hangzhou', 'China', 30.25, 120.17),
                 48: ('Harbin', 'Harbin', 'China', 45.75, 126.65),
                 49: ('Hebi', 'Hebi', 'China', 35.95, 114.22),
                 50: ('Hefei', 'Hefei', 'China', 31.85, 117.28),
                 51: ('Hegang', 'Hegang', 'China', 47.4, 130.37),
                 52: ('Hengshui', 'Hengshui', 'China', 37.72, 115.7),
                 53: ('Hengyang', 'Hengyang', 'China', 26.89, 112.62),
                 54: ('Heze', 'Heze', 'China', 35.23, 115.45),
                 55: ('Hohhot', 'Hohhot', 'China', 40.82, 111.64),
                 56: ('Huaibei', 'Huaibei', 'China', 33.95, 116.75),
                 57: ('Huainan', 'Huainan', 'China', 32.63, 116.98),
                 58: ('Huaiyin', 'Huaiyin', 'China', 33.58, 119.03),
                 59: ('Huanggang', 'Huanggang', 'China', 23.68, 117.0),
                 60: ('Huangshi', 'Huangshi', 'China', 30.22, 115.1),
                 61: ('Huizhou', 'Huizhou', 'China', 23.08, 114.4),
                 62: ('Hulan Ergi', 'Hulan Ergi', 'China', 47.21, 123.61),
                 63: ('Jiamusi', 'Jiamusi', 'China', 46.83, 130.35),
                 64: ('Jiangmen', 'Jiangmen', 'China', 22.58, 113.08),
                 65: ('Jiaojiang', 'Jiaojiang', 'China', 28.68, 121.45),
                 66: ('Jiaozuo', 'Jiaozuo', 'China', 35.25, 113.22),
                 67: ('Jiaxing', 'Jiaxing', 'China', 30.77, 120.75),
                 68: ('Jilin', 'Jilin', 'China', 43.85, 126.55),
                 69: ('Jinan', 'Jinan', 'China', 36.67, 117.0),
                 70: ('Jincheng', 'Jincheng', 'China', 35.5, 112.83),
                 71: ('Jingdezhen', 'Jingdezhen', 'China', 29.27, 117.18),
                 72: ('Jining', 'Jining', 'China', 35.4, 116.55),
                 73: ('Jining', 'Jining', 'China', 41.04, 113.09),
                 74: ('Jinxi', 'Jinxi', 'China', 40.75, 120.83),
                 75: ('Jinzhou', 'Jinzhou', 'China', 41.12, 121.1),
                 76: ('Jiujiang', 'Jiujiang', 'China', 29.73, 115.98),
                 77: ('Jiulong', 'Jiulong', 'China', 22.32, 114.17),
                 78: ('Jixi', 'Jixi', 'China', 45.3, 130.97),
                 79: ('Kaifeng', 'Kaifeng', 'China', 34.85, 114.35),
                 80: ('Kashi', 'Kashi', 'China', 39.48, 75.97),
                 82: ('Kunming', 'Kunming', 'China', 25.05, 102.7),
                 83: ('Kuytun', 'Kuytun', 'China', 44.44, 84.89),
                 84: ('Langfang', 'Langfang', 'China', 39.52, 116.68),
                 85: ('Lanzhou', 'Lanzhou', 'China', 36.05, 103.68),
                 86: ('Laohekou', 'Laohekou', 'China', 32.39, 111.66),
                 87: ('Liaoyang', 'Liaoyang', 'China', 41.28, 123.18),
                 88: ('Liaoyuan', 'Liaoyuan', 'China', 42.9, 125.13),
                 89: ('Linyi', 'Linyi', 'China', 35.08, 118.33),
                 90: ('Liuzhou', 'Liuzhou', 'China', 24.28, 109.25),
                 91: ('Luancheng', 'Luancheng', 'China', 37.88, 114.65),
                 92: ('Luohe', 'Luohe', 'China', 33.57, 114.03),
                 93: ('Luoyang', 'Luoyang', 'China', 34.68, 112.47),
                 94: ('Luqiao', 'Luqiao', 'China', 28.58, 121.38),
                 95: ('Luzhou', 'Luzhou', 'China', 28.88, 105.38),
                 96: ('Maanshan', 'Maanshan', 'China', 31.73, 118.48),
                 97: ('Maoming', 'Maoming', 'China', 21.92, 110.87),
                 98: ('Mianchang', 'Mianchang', 'China', 23.26, 116.59),
                 99: ('Mianyang', 'Mianyang', 'China', 31.47, 104.77),
                 100: ('Mudanjiang', 'Mudanjiang', 'China', 44.58, 129.6),
                 101: ('Nanchang', 'Nanchang', 'China', 28.68, 115.88),
                 102: ('Nanjing', 'Nanjing', 'China', 32.05, 118.78),
                 103: ('Nanning', 'Nanning', 'China', 22.82, 108.32),
                 104: ('Nantong', 'Nantong', 'China', 32.02, 120.82),
                 105: ('Neijiang', 'Neijiang', 'China', 29.58, 105.05),
                 106: ('Ningbo', 'Ningbo', 'China', 29.88, 121.55),
                 107: ('Panjin', 'Panjin', 'China', 41.18, 122.05),
                 108: ('Panzhihua', 'Panzhihua', 'China', 26.58, 101.68),
                 109: ('Peking', 'Peking', 'China', 39.93, 116.4),
                 110: ('Pingdingshan', 'Pingdingshan', 'China', 33.73, 113.3),
                 111: ('Pingxiang', 'Pingxiang', 'China', 27.62, 113.85),
                 112: ('Putian', 'Putian', 'China', 25.43, 119.02),
                 113: ('Puyang', 'Puyang', 'China', 35.7, 114.98),
                 114: ('Qingdao', 'Qingdao', 'China', 36.07, 120.32),
                 115: ('Qinhuangdao', 'Qinhuangdao', 'China', 39.93, 119.62),
                 116: ('Qiqihar', 'Qiqihar', 'China', 47.35, 124.0),
                 117: ('Qitaihe', 'Qitaihe', 'China', 45.8, 130.85),
                 118: ('Quanwan', 'Quanwan', 'China', 22.37, 114.11),
                 119: ('Rongcheng', 'Rongcheng', 'China', 23.54, 116.34),
                 121: ('Sanmenxia', 'Sanmenxia', 'China', 34.83, 111.08),
                 122: ('Shanghai', 'Shanghai', 'China', 31.23, 121.47),
                 123: ('Shangrao', 'Shangrao', 'China', 28.47, 117.97),
                 124: ('Shantou', 'Shantou', 'China', 23.37, 116.67),
                 125: ('Shaoguan', 'Shaoguan', 'China', 24.8, 113.58),
                 126: ('Shaoxing', 'Shaoxing', 'China', 30.0, 120.57),
                 127: ('Shaoyang', 'Shaoyang', 'China', 27.0, 111.2),
                 128: ('Shashi', 'Shashi', 'China', 30.32, 112.23),
                 129: ('Shatian', 'Shatian', 'China', 22.38, 114.19),
                 130: ('Shenyang', 'Shenyang', 'China', 41.8, 123.45),
                 131: ('Shenzhen', 'Shenzhen', 'China', 22.53, 114.13),
                 132: ('Shihezi', 'Shihezi', 'China', 44.3, 86.03),
                 133: ('Shijiazhuang', 'Shijiazhuang', 'China', 38.05, 114.48),
                 134: ('Shiongshui', 'Shiongshui', 'China', 22.52, 114.12),
                 136: ('Shiyan', 'Shiyan', 'China', 32.57, 110.78),
                 137: ('Siping', 'Siping', 'China', 43.17, 124.33),
                 138: ('Sucheng', 'Sucheng', 'China', 33.95, 118.29),
                 139: ('Suihua', 'Suihua', 'China', 46.63, 126.98),
                 140: ('Suizhou', 'Suizhou', 'China', 31.71, 113.36),
                 141: ('Suzhou', 'Suzhou', 'China', 31.3, 120.62),
                 142: ('Taian', 'Taian', 'China', 36.2, 117.12),
                 143: ('Taiyuan', 'Taiyuan', 'China', 37.87, 112.55),
                 144: ('Taizhou', 'Taizhou', 'China', 32.49, 119.9),
                 145: ('Tanggu', 'Tanggu', 'China', 39.0, 117.67),
                 146: ('Tangshan', 'Tangshan', 'China', 39.62, 118.19),
                 147: ('Tianjin', 'Tianjin', 'China', 39.13, 117.2),
                 148: ('Tieling', 'Tieling', 'China', 42.3, 123.82),
                 149: ('Tongliao', 'Tongliao', 'China', 43.62, 122.27),
                 150: ('Tongling', 'Tongling', 'China', 30.95, 117.78),
                 151: ('Urumqi', 'Urumqi', 'China', 43.8, 87.58),
                 152: ('Weifang', 'Weifang', 'China', 36.72, 119.1),
                 153: ('Wenzhou', 'Wenzhou', 'China', 28.02, 120.65),
                 154: ('Wuhan', 'Wuhan', 'China', 30.58, 114.27),
                 155: ('Wuhu', 'Wuhu', 'China', 31.35, 118.37),
                 156: ('Wuxi', 'Wuxi', 'China', 31.58, 120.3),
                 157: ('Wuzhou', 'Wuzhou', 'China', 23.48, 111.32),
                 158: ('Xiamen', 'Xiamen', 'China', 24.45, 118.08),
                 159: ('Xian', 'Xian', 'China', 34.27, 108.9),
                 160: ('Xiangfan', 'Xiangfan', 'China', 32.02, 112.13),
                 161: ('Xianggangdao', 'Xianggangdao', 'China', 22.27, 114.14),
                 162: ('Xiangtan', 'Xiangtan', 'China', 27.85, 112.9),
                 163: ('Xianyang', 'Xianyang', 'China', 34.37, 108.7),
                 164: ('Xigong', 'Xigong', 'China', 22.33, 114.25),
                 165: ('Xingtai', 'Xingtai', 'China', 37.07, 114.49),
                 166: ('Xining', 'Xining', 'China', 36.62, 101.77),
                 167: ('Xinpu', 'Xinpu', 'China', 34.6, 119.17),
                 168: ('Xinxiang', 'Xinxiang', 'China', 35.32, 113.87),
                 169: ('Xinyang', 'Xinyang', 'China', 32.13, 114.07),
                 170: ('Xuanhua', 'Xuanhua', 'China', 40.6, 115.03),
                 171: ('Xuchang', 'Xuchang', 'China', 34.02, 113.82),
                 172: ('Xuzhou', 'Xuzhou', 'China', 34.27, 117.18),
                 173: ('Yancheng', 'Yancheng', 'China', 33.39, 120.12),
                 174: ('Yangjiang', 'Yangjiang', 'China', 21.85, 111.97),
                 175: ('Yangquan', 'Yangquan', 'China', 37.87, 113.57),
                 176: ('Yangzhou', 'Yangzhou', 'China', 32.4, 119.43),
                 177: ('Yanji', 'Yanji', 'China', 42.88, 129.52),
                 178: ('Yantai', 'Yantai', 'China', 37.53, 121.4),
                 179: ('Yichang', 'Yichang', 'China', 30.7, 111.28),
                 180: ('Yinchuan', 'Yinchuan', 'China', 38.47, 106.32),
                 181: ('Yingcheng', 'Yingcheng', 'China', 30.94, 113.57),
                 182: ('Yingkou', 'Yingkou', 'China', 40.67, 122.28),
                 183: ('Yining', 'Yining', 'China', 43.9, 81.35),
                 184: ('Yuanlong', 'Yuanlong', 'China', 22.44, 114.02),
                 185: ('Yueyang', 'Yueyang', 'China', 29.38, 113.1),
                 186: ('Zhangdian', 'Zhangdian', 'China', 36.8, 118.06),
                 187: ('Zhangjiakou', 'Zhangjiakou', 'China', 40.83, 114.93),
                 188: ('Zhangzhou', 'Zhangzhou', 'China', 24.52, 117.67),
                 189: ('Zhanjiang', 'Zhanjiang', 'China', 21.2, 110.38),
                 190: ('Zhaoqing', 'Zhaoqing', 'China', 23.05, 112.45),
                 191: ('Zhengzhou', 'Zhengzhou', 'China', 34.75, 113.67),
                 192: ('Zhenjiang', 'Zhenjiang', 'China', 32.22, 119.43),
                 193: ('Zhongshan', 'Zhongshan', 'China', 22.53, 113.35),
                 194: ('Zhoukou', 'Zhoukou', 'China', 33.64, 114.63),
                 195: ('Zhuhai', 'Zhuhai', 'China', 22.28, 113.57),
                 196: ('Zhumadian', 'Zhumadian', 'China', 32.97, 114.05),
                 197: ('Zhunmen', 'Zhunmen', 'China', 22.41, 113.98),
                 198: ('Zhuzhou', 'Zhuzhou', 'China', 27.83, 113.15),
                 199: ('Zigong', 'Zigong', 'China', 29.4, 104.78),
                 200: ('Zunyi', 'Zunyi', 'China', 27.7, 106.92)}
    
    # customer_demands: {id: demand_value}
    customer_demands = {1: 458287, 2: 379701, 3: 1192903, 4: 448957, 5: 817193, 6: 516212, 7: 351212, 8: 1141567, 9: 472186, 10: 1363141, 11: 368271, 12: 606043, 13: 1083983, 14: 605426, 15: 2813245, 16: 599402, 17: 2323490, 18: 730188, 19: 1000831, 20: 500792, 21: 520199, 22: 464916, 23: 3917204, 24: 391521, 25: 4757368, 26: 355037, 27: 374588, 28: 2195151, 29: 673737, 30: 1071836, 31: 464914, 32: 414683, 33: 405576, 34: 314933, 35: 811598, 36: 1417054, 37: 702802, 38: 1265124, 39: 267242, 40: 3097347, 41: 741605, 42: 1203454, 43: 257460, 44: 680177, 45: 1384575, 46: 253323, 47: 1920846, 48: 3405339, 49: 254512, 50: 1447370, 51: 778737, 52: 491221, 53: 753745, 54: 257160, 55: 802403, 56: 1035805, 57: 1264768, 58: 557565, 59: 253134, 60: 718104, 61: 320049, 62: 252970, 63: 561974, 64: 631850, 65: 481402, 66: 547871, 67: 476703, 68: 2228535, 69: 2227937, 70: 449049, 71: 311728, 72: 519002, 73: 275344, 74: 311278, 75: 588094, 76: 253448, 77: 1929557, 78: 484961, 79: 585908, 80: 321042, 82: 1105391, 83: 257485, 84: 734039, 85: 1449972, 86: 262644, 87: 742723, 88: 468416, 89: 277957, 90: 791427, 91: 608672, 92: 516955, 93: 1391054, 94: 437622, 95: 292765, 96: 555686, 97: 663707, 98: 359317, 99: 271695, 100: 700538,
                        101: 2156849, 102: 3389828, 103: 795281, 104: 669054, 105: 542253, 106: 736240, 107: 685122, 108: 457574, 109: 7923530, 110: 889978, 111: 371596, 112: 387764, 113: 823245, 114: 1736114, 115: 833239, 116: 924737, 117: 340236, 118: 266055, 119: 1586793, 121: 494772, 122: 16348947, 123: 412553, 124: 1493452, 125: 762371, 126: 531398, 127: 674835, 128: 570044, 129: 693616, 130: 3559349, 131: 1370396, 132: 606906, 133: 2415478, 134: 565146, 136: 426017, 137: 649152, 138: 455710, 139: 252734, 140: 353705, 141: 1606518, 142: 933760, 143: 2809501, 144: 614932, 145: 597322, 146: 1627819, 147: 3649387, 148: 350640, 149: 270095, 150: 324728, 151: 1628533, 152: 422040, 153: 885362, 154: 4343546, 155: 513241, 156: 1243036, 157: 271835, 158: 636548, 159: 4587818, 160: 468606, 161: 1309560, 162: 668990, 163: 1118112, 164: 604899, 165: 748789, 166: 765485, 167: 490649, 168: 743854, 169: 2031326, 170: 473281, 171: 557220, 172: 1336626, 173: 631085, 174: 445158, 175: 433675, 176: 541986, 177: 360540, 178: 811224, 179: 483391, 180: 492741, 181: 272961, 182: 623451, 183: 282156, 184: 696334, 185: 524068, 186: 674380, 187: 857555, 188: 683716, 189: 729140, 190: 405243, 191: 2014810, 192: 635213, 193: 355546, 194: 420571, 195: 554443, 196: 315210, 197: 532585, 198: 703888, 199: 684157, 200: 554597}

    # distance: {(from, to): distance}
    distance = {(15, 1): 3660.0, (15, 2): 1655.0, (15, 3): 363.0, (15, 4): 2623.0, (15, 5): 1273.0, (15, 6): 2633.0, (15, 7): 279.0, (15, 8): 992.0, (15, 9): 1884.0, (15, 10): 1308.0, (15, 11): 2903.0, (15, 12): 1398.0, (15, 13): 311.0, (15, 14): 940.0, (15, 15): 0.0, (15, 16): 2046.0, (15, 17): 2061.0, (15, 18): 1507.0, (15, 19): 1423.0, (15, 20): 478.0, (15, 21): 2381.0, (15, 22): 689.0, (15, 23): 2374.0, (15, 24): 549.0, (15, 25): 2296.0, (15, 26): 1424.0, (15, 27): 2592.0, (15, 28): 630.0, (15, 29): 423.0, (15, 30): 1080.0, (15, 31): 1043.0, (15, 32): 242.0, (15, 33): 2546.0, (15, 34): 912.0, (15, 35): 2572.0, (15, 36): 252.0, (15, 37): 365.0, (15, 38): 2051.0, (15, 39): 2202.0, (15, 40): 2557.0, (15, 41): 2475.0, (15, 42): 2547.0, (15, 43): 1603.0, (15, 44): 2991.0, (15, 45): 1226.0, (15, 46): 813.0, (15, 47): 1581.0, (15, 48): 233.0, (15, 49): 1292.0, (15, 50): 1510.0, (15, 51): 553.0, (15, 52): 1060.0, (15, 53): 2205.0, (15, 54): 1279.0, (15, 55): 1174.0, (15, 56): 1328.0, (15, 57): 1445.0, (15, 58): 1267.0, (15, 59): 2369.0, (15, 60): 1765.0, (15, 61): 2518.0, (15, 62): 395.0, (15, 63): 510.0, (15, 64): 2619.0, (15, 65): 1723.0, (15, 66): 1410.0, (15, 67): 1511.0, (15, 68): 96.0, (15, 69): 1067.0, (15, 70): 1415.0, (15, 71): 1776.0, (15, 72): 1204.0, (15, 73): 1052.0, (15, 74): 508.0, (15, 75): 463.0, (15, 76): 1776.0, (15, 77): 2605.0, (15, 78): 472.0, (15, 79): 1375.0, (15, 80): 4066.0, (15, 82): 2927.0, (15, 83): 3193.0, (15, 84): 866.0, (15, 85): 2032.0, (15, 86): 1744.0, (15, 87): 338.0, (15, 88): 109.0, (15, 89): 1146.0, (15, 90): 2623.0, (15, 91): 1117.0, (15, 92): 1505.0, (15, 93): 1503.0, (15, 94): 1735.0, (15, 95): 2430.0, (15, 96): 1476.0, (15, 97): 2778.0, (15, 98): 2426.0, (15, 99): 2264.0, (15, 100): 347.0, (15, 101): 1886.0, (15, 102): 1433.0, (15, 103): 2810.0, (15, 104): 1375.0, (15, 105): 2394.0, (15, 106): 1590.0, (15, 107): 403.0, (15, 108): 2863.0, (15, 109): 859.0, (15, 110): 1532.0, (15, 111): 2078.0, (15, 112): 2127.0, (15, 113): 1266.0, (15, 114): 966.0, (15, 115): 645.0, (15, 116): 401.0, (15, 117): 483.0, (15, 118): 2602.0, (15, 119): 2404.0, (15, 121): 1581.0, (15, 122): 1445.0, (15, 123): 1833.0, (15, 124): 2412.0, (15, 125): 2373.0, (15, 126): 1598.0, (15, 127): 2263.0, (15, 128): 1897.0, (15, 129): 2598.0, (15, 130): 277.0, (15, 131): 2585.0, (15, 132): 3108.0, (15, 133): 1117.0, (15, 134): 2586.0, (15, 136): 1782.0, (15, 137): 113.0, (15, 138): 1259.0, (15, 139): 332.0, (15, 140): 1709.0, (15, 141): 1457.0, (15, 142): 1102.0, (15, 143): 1263.0, (15, 144): 1350.0, (15, 145): 837.0, (15, 146): 758.0, (15, 147): 858.0, (15, 148): 214.0, (15, 149): 249.0, (15, 150): 1582.0, (15, 151): 3000.0, (15, 152): 954.0, (15, 153): 1810.0, (15, 154): 1769.0, (15, 155): 1519.0, (15, 156): 1435.0, (15, 157): 2602.0, (15, 158): 2257.0, (15, 159): 1770.0, (15, 160): 1749.0, (15, 161): 2611.0, (15, 162): 2098.0, (15, 163): 1776.0, (15, 164): 2601.0, (15, 165): 1187.0, (15, 166): 2147.0, (15, 167): 1158.0, (15, 168): 1365.0, (15, 169): 1633.0, (15, 170): 923.0, (15, 171): 1477.0, (15, 172): 1277.0, (15, 173): 1249.0, (15, 174): 2739.0, (15, 175): 1192.0, (15, 176): 1375.0, (15, 177): 354.0, (15, 178): 779.0, (15, 179): 1915.0, (15, 180): 1696.0, (15, 181): 1770.0, (15, 182): 436.0, (15, 183): 3481.0, (15, 184): 2598.0, (15, 185): 1941.0, (15, 186): 998.0, (15, 187): 919.0, (15, 188): 2260.0, (15, 189): 2872.0, (15, 190): 2597.0, (15, 191): 1424.0, (15, 192): 1393.0, (15, 193): 2614.0, (15, 194): 1465.0, (15, 195): 2631.0, (15, 196): 1557.0, (15, 197): 2603.0, (15, 198): 2089.0, (15, 199): 2427.0, (15, 200): 2436.0, (17, 1): 3292.0, (17, 2): 471.0, (17, 3): 1698.0, (17, 4): 728.0, (17, 5): 885.0, (17, 6): 662.0, (17, 7): 2120.0, (17, 8): 1208.0, (17, 9): 881.0, (17, 10): 1403.0, (17, 11): 842.0, (17, 12): 673.0, (17, 13): 1757.0, (17, 14): 1181.0, (17, 15): 2061.0, (17, 16): 156.0, (17, 17): 0.0, (17, 18): 788.0, (17, 19): 782.0, (17, 20): 1630.0, (17, 21): 623.0, (17, 22): 1488.0, (17, 23): 904.0, (17, 24): 1654.0, (17, 25): 640.0, (17, 26): 685.0, (17, 27): 650.0, (17, 28): 1435.0, (17, 29): 1688.0, (17, 30): 1320.0, (17, 31): 1074.0, (17, 32): 1907.0, (17, 33): 577.0, (17, 34): 1155.0, (17, 35): 575.0, (17, 36): 1811.0, (17, 37): 1723.0, (17, 38): 669.0, (17, 39): 316.0, (17, 40): 565.0, (17, 41): 420.0, (17, 42): 642.0, (17, 43): 1394.0, (17, 44): 945.0, (17, 45): 942.0, (17, 46): 1303.0, (17, 47): 734.0, (17, 48): 2290.0, (17, 49): 869.0, (17, 50): 580.0, (17, 51): 2611.0, (17, 52): 1088.0, (17, 53): 150.0, (17, 54): 815.0, (17, 55): 1407.0, (17, 56): 733.0, (17, 57): 624.0, (17, 58): 831.0, (17, 59): 644.0, (17, 60): 305.0, (17, 61): 587.0, (17, 62): 2305.0, (17, 63): 2563.0, (17, 64): 625.0, (17, 65): 830.0, (17, 66): 784.0, (17, 67): 805.0, (17, 68): 2118.0, (17, 69): 1014.0, (17, 70): 811.0, (17, 71): 427.0, (17, 72): 868.0, (17, 73): 1427.0, (17, 74): 1568.0, (17, 75): 1615.0, (17, 76): 338.0, (17, 77): 664.0, (17, 78): 2474.0, (17, 79): 750.0, (17, 80): 3607.0, (17, 82): 1078.0, (17, 83): 3069.0, (17, 84): 1303.0, (17, 85): 1234.0, (17, 86): 482.0, (17, 87): 1724.0, (17, 88): 1964.0, (17, 89): 917.0, (17, 90): 572.0, (17, 91): 1087.0, (17, 92): 605.0, (17, 93): 722.0, (17, 94): 823.0, (17, 95): 745.0, (17, 96): 660.0, (17, 97): 729.0, (17, 98): 658.0, (17, 99): 870.0, (17, 100): 2341.0, (17, 101): 289.0, (17, 102): 703.0, (17, 103): 758.0, (17, 104): 865.0, (17, 105): 785.0, (17, 106): 854.0, (17, 107): 1662.0, (17, 108): 1128.0, (17, 109): 1341.0, (17, 110): 615.0, (17, 111): 108.0, (17, 112): 674.0, (17, 113): 855.0, (17, 114): 1114.0, (17, 115): 1439.0, (17, 116): 2332.0, (17, 117): 2506.0, (17, 118): 658.0, (17, 119): 618.0, (17, 121): 758.0, (17, 122): 886.0, (17, 123): 490.0, (17, 124): 652.0, (17, 125): 383.0, (17, 126): 764.0, (17, 127): 219.0, (17, 128): 246.0, (17, 129): 658.0, (17, 130): 1784.0, (17, 131): 641.0, (17, 132): 2981.0, (17, 133): 1103.0, (17, 134): 642.0, (17, 136): 529.0, (17, 137): 1950.0, (17, 138): 815.0, (17, 139): 2384.0, (17, 140): 392.0, (17, 141): 814.0, (17, 142): 970.0, (17, 143): 1075.0, (17, 144): 817.0, (17, 145): 1276.0, (17, 146): 1356.0, (17, 147): 1276.0, (17, 148): 1847.0, (17, 149): 1904.0, (17, 150): 556.0, (17, 151): 2846.0, (17, 152): 1107.0, (17, 153): 753.0, (17, 154): 293.0, (17, 155): 627.0, (17, 156): 800.0, (17, 157): 550.0, (17, 158): 658.0, (17, 159): 777.0, (17, 160): 432.0, (17, 161): 669.0, (17, 162): 39.0, (17, 163): 796.0, (17, 164): 665.0, (17, 165): 996.0, (17, 166): 1405.0, (17, 167): 922.0, (17, 168): 796.0, (17, 169): 449.0, (17, 170): 1391.0, (17, 171): 652.0, (17, 172): 784.0, (17, 173): 893.0, (17, 174): 713.0, (17, 175): 1076.0, (17, 176): 776.0, (17, 177): 2205.0, (17, 178): 1300.0, (17, 179): 322.0, (17, 180): 1296.0, (17, 181): 310.0, (17, 182): 1625.0, (17, 183): 3303.0, (17, 184): 649.0, (17, 185): 132.0, (17, 186): 1068.0, (17, 187): 1415.0, (17, 188): 621.0, (17, 189): 820.0, (17, 190): 575.0, (17, 191): 731.0, (17, 192): 764.0, (17, 193): 631.0, (17, 194): 625.0, (17, 195): 661.0, (17, 196): 540.0, (17, 197): 651.0, (17, 198): 45.0, (17, 199): 808.0, (17, 200): 596.0,
                (23, 1): 2428.0, (23, 2): 1241.0, (23, 3): 2051.0, (23, 4): 524.0, (23, 5): 1126.0, (23, 6): 1326.0, (23, 7): 2322.0, (23, 8): 1382.0, (23, 9): 503.0, (23, 10): 1228.0, (23, 11): 1137.0, (23, 12): 1277.0, (23, 13): 2121.0, (23, 14): 1446.0, (23, 15): 2374.0, (23, 16): 756.0, (23, 17): 904.0, (23, 18): 876.0, (23, 19): 1514.0, (23, 20): 1896.0, (23, 21): 1464.0, (23, 22): 1689.0, (23, 23): 0.0, (23, 24): 1847.0, (23, 25): 270.0, (23, 26): 1359.0, (23, 27): 1355.0, (23, 28): 1843.0, (23, 29): 2113.0, (23, 30): 1337.0, (23, 31): 1353.0, (23, 32): 2303.0, (23, 33): 1277.0, (23, 34): 1528.0, (23, 35): 1234.0, (23, 36): 2159.0, (23, 37): 2009.0, (23, 38): 1572.0, (23, 39): 1184.0, (23, 40): 1236.0, (23, 41): 854.0, (23, 42): 523.0, (23, 43): 1029.0, (23, 44): 1336.0, (23, 45): 1165.0, (23, 46): 1567.0, (23, 47): 1541.0, (23, 48): 2571.0, (23, 49): 1109.0, (23, 50): 1261.0, (23, 51): 2909.0, (23, 52): 1323.0, (23, 53): 932.0, (23, 54): 1175.0, (23, 55): 1317.0, (23, 56): 1244.0, (23, 57): 1240.0, (23, 58): 1443.0, (23, 59): 1494.0, (23, 60): 1057.0, (23, 61): 1325.0, (23, 62): 2482.0, (23, 63): 2876.0, (23, 64): 1267.0, (23, 65): 1691.0, (23, 66): 993.0, (23, 67): 1592.0, (23, 68): 2456.0, (23, 69): 1367.0, (23, 70): 976.0, (23, 71): 1271.0, (23, 72): 1275.0, (23, 73): 1408.0, (23, 74): 1876.0, (23, 75): 1917.0, (23, 76): 1148.0, (23, 77): 1366.0, (23, 78): 2840.0, (23, 79): 1066.0, (23, 80): 2723.0, (23, 82): 639.0, (23, 83): 2269.0, (23, 84): 1508.0, (23, 85): 599.0, (23, 86): 744.0, (23, 87): 2076.0, (23, 88): 2304.0, (23, 89): 1416.0, (23, 90): 874.0, (23, 91): 1258.0, (23, 92): 991.0, (23, 93): 903.0, (23, 94): 1686.0, (23, 95): 236.0, (23, 96): 1374.0, (23, 97): 1184.0, (23, 98): 1487.0, (23, 99): 111.0, (23, 100): 2709.0, (23, 101): 1161.0, (23, 102): 1403.0, (23, 103): 969.0, (23, 104): 1595.0, (23, 105): 153.0, (23, 106): 1678.0, (23, 107): 1989.0, (23, 108): 511.0, (23, 109): 1517.0, (23, 110): 932.0, (23, 111): 1007.0, (23, 112): 1576.0, (23, 113): 1157.0, (23, 114): 1620.0, (23, 115): 1741.0, (23, 116): 2514.0, (23, 117): 2856.0, (23, 118): 1358.0, (23, 119): 1448.0, (23, 121): 801.0, (23, 122): 1658.0, (23, 123): 1364.0, (23, 124): 1486.0, (23, 125): 1139.0, (23, 126): 1583.0, (23, 127): 805.0, (23, 128): 782.0, (23, 129): 1363.0, (23, 130): 2124.0, (23, 131): 1347.0, (23, 132): 2187.0, (23, 133): 1257.0, (23, 134): 1347.0, (23, 136): 669.0, (23, 137): 2263.0, (23, 138): 1383.0, (23, 139): 2646.0, (23, 140): 890.0, (23, 141): 1577.0, (23, 142): 1355.0, (23, 143): 1115.0, (23, 144): 1511.0, (23, 145): 1545.0, (23, 146): 1620.0, (23, 147): 1519.0, (23, 148): 2178.0, (23, 149): 2152.0, (23, 150): 1308.0, (23, 151): 2056.0, (23, 152): 1541.0, (23, 153): 1631.0, (23, 154): 975.0, (23, 155): 1363.0, (23, 156): 1546.0, (23, 157): 1073.0, (23, 158): 1542.0, (23, 159): 604.0, (23, 160): 779.0, (23, 161): 1368.0, (23, 162): 911.0, (23, 163): 598.0, (23, 164): 1371.0, (23, 165): 1194.0, (23, 166): 694.0, (23, 167): 1477.0, (23, 168): 1049.0, (23, 169): 962.0, (23, 170): 1480.0, (23, 171): 988.0, (23, 172): 1291.0, (23, 173): 1540.0, (23, 174): 1256.0, (23, 175): 1182.0, (23, 176): 1466.0, (23, 177): 2625.0, (23, 178): 1764.0, (23, 179): 689.0, (23, 180): 891.0, (23, 181): 907.0, (23, 182): 1978.0, (23, 183): 2475.0, (23, 184): 1346.0, (23, 185): 880.0, (23, 186): 1459.0, (23, 187): 1492.0, (23, 188): 1502.0, (23, 189): 1226.0, (23, 190): 1185.0, (23, 191): 1005.0, (23, 192): 1465.0, (23, 193): 1290.0, (23, 194): 1046.0, (23, 195): 1326.0, (23, 196): 976.0, (23, 197): 1345.0, (23, 198): 935.0, (23, 199): 157.0, (23, 200): 430.0, (25, 1): 2695.0, (25, 2): 1012.0, (25, 3): 1956.0, (25, 4): 374.0, (25, 5): 1024.0, (25, 6): 1069.0, (25, 7): 2276.0, (25, 8): 1316.0, (25, 9): 537.0, (25, 10): 1265.0, (25, 11): 934.0, (25, 12): 1087.0, (25, 13): 2024.0, (25, 14): 1356.0, (25, 15): 2296.0, (25, 16): 498.0, (25, 17): 640.0, (25, 18): 793.0, (25, 19): 1302.0, (25, 20): 1822.0, (25, 21): 1194.0, (25, 22): 1629.0, (25, 23): 270.0, (25, 24): 1793.0, (25, 25): 0.0, (25, 26): 1157.0, (25, 27): 1094.0, (25, 28): 1727.0, (25, 29): 1998.0, (25, 30): 1318.0, (25, 31): 1255.0, (25, 32): 2200.0, (25, 33): 1015.0, (25, 34): 1412.0, (25, 35): 976.0, (25, 36): 2067.0, (25, 37): 1932.0, (25, 38): 1308.0, (25, 39): 914.0, (25, 40): 977.0, (25, 41): 600.0, (25, 42): 333.0, (25, 43): 1121.0, (25, 44): 1123.0, (25, 45): 1070.0, (25, 46): 1483.0, (25, 47): 1310.0, (25, 48): 2506.0, (25, 49): 1005.0, (25, 50): 1053.0, (25, 51): 2843.0, (25, 52): 1236.0, (25, 53): 662.0, (25, 54): 1042.0, (25, 55): 1331.0, (25, 56): 1076.0, (25, 57): 1046.0, (25, 58): 1259.0, (25, 59): 1224.0, (25, 60): 824.0, (25, 61): 1061.0, (25, 62): 2446.0, (25, 63): 2805.0, (25, 64): 1011.0, (25, 65): 1446.0, (25, 66): 886.0, (25, 67): 1367.0, (25, 68): 2372.0, (25, 69): 1249.0, (25, 70): 881.0, (25, 71): 1026.0, (25, 72): 1136.0, (25, 73): 1403.0, (25, 74): 1790.0, (25, 75): 1833.0, (25, 76): 908.0, (25, 77): 1106.0, (25, 78): 2753.0, (25, 79): 936.0, (25, 80): 2993.0, (25, 82): 632.0, (25, 83): 2522.0, (25, 84): 1439.0, (25, 85): 769.0, (25, 86): 576.0, (25, 87): 1982.0, (25, 88): 2217.0, (25, 89): 1260.0, (25, 90): 644.0, (25, 91): 1186.0, (25, 92): 833.0, (25, 93): 793.0, (25, 94): 1440.0, (25, 95): 139.0, (25, 96): 1162.0, (25, 97): 952.0, (25, 98): 1217.0, (25, 99): 273.0, (25, 100): 2619.0, (25, 101): 908.0, (25, 102): 1196.0, (25, 103): 770.0, (25, 104): 1385.0, (25, 105): 148.0, (25, 106): 1444.0, (25, 107): 1899.0, (25, 108): 584.0, (25, 109): 1457.0, (25, 110): 786.0, (25, 111): 741.0, (25, 112): 1309.0, (25, 113): 1039.0, (25, 114): 1470.0, (25, 115): 1653.0, (25, 116): 2478.0, (25, 117): 2774.0, (25, 118): 1097.0, (25, 119): 1178.0, (25, 121): 721.0, (25, 122): 1438.0, (25, 123): 1113.0, (25, 124): 1216.0, (25, 125): 871.0, (25, 126): 1349.0, (25, 127): 535.0, (25, 128): 550.0, (25, 129): 1102.0, (25, 130): 2034.0, (25, 131): 1086.0, (25, 132): 2439.0, (25, 133): 1190.0, (25, 134): 1086.0, (25, 136): 520.0, (25, 137): 2183.0, (25, 138): 1208.0, (25, 139): 2587.0, (25, 140): 690.0, (25, 141): 1358.0, (25, 142): 1227.0, (25, 143): 1074.0, (25, 144): 1308.0, (25, 145): 1459.0, (25, 146): 1538.0, (25, 147): 1439.0, (25, 148): 2092.0, (25, 149): 2089.0, (25, 150): 1085.0, (25, 151): 2306.0, (25, 152): 1408.0, (25, 153): 1380.0, (25, 154): 748.0, (25, 155): 1146.0, (25, 156): 1330.0, (25, 157): 824.0, (25, 158): 1272.0, (25, 159): 566.0, (25, 160): 595.0, (25, 161): 1108.0, (25, 162): 645.0, (25, 163): 570.0, (25, 164): 1110.0, (25, 165): 1110.0, (25, 166): 902.0, (25, 167): 1309.0, (25, 168): 935.0, (25, 169): 769.0, (25, 170): 1445.0, (25, 171): 843.0, (25, 172): 1127.0, (25, 173): 1350.0, (25, 174): 1013.0, (25, 175): 1125.0, (25, 176): 1263.0, (25, 177): 2519.0, (25, 178): 1630.0, (25, 179): 469.0, (25, 180): 989.0, (25, 181): 688.0, (25, 182): 1881.0, (25, 183): 2735.0, (25, 184): 1086.0, (25, 185): 631.0, (25, 186): 1334.0, (25, 187): 1461.0, (25, 188): 1232.0, (25, 189): 1005.0, (25, 190): 931.0, (25, 191): 880.0, (25, 192): 1259.0, (25, 193): 1033.0, (25, 194): 885.0, (25, 195): 1069.0, (25, 196): 803.0, (25, 197): 1085.0, (25, 198): 669.0, (25, 199): 175.0, (25, 200): 210.0, 
                (28, 1): 3498.0, (28, 2): 1025.0, (28, 3): 268.0, (28, 4): 2031.0, (28, 5): 716.0, (28, 6): 2003.0, (28, 7): 751.0, (28, 8): 534.0, (28, 9): 1386.0, (28, 10): 1007.0, (28, 11): 2277.0, (28, 12): 769.0, (28, 13): 322.0, (28, 14): 420.0, (28, 15): 630.0, (28, 16): 1430.0, (28, 17): 1435.0, (28, 18): 968.0, (28, 19): 808.0, (28, 20): 310.0, (28, 21): 1759.0, (28, 22): 390.0, (28, 23): 1843.0, (28, 24): 436.0, (28, 25): 1727.0, (28, 26): 795.0, (28, 27): 1963.0, (28, 28): 0.0, (28, 29): 271.0, (28, 30): 727.0, (28, 31): 495.0, (28, 32): 478.0, (28, 33): 1916.0, (28, 34): 316.0, (28, 35): 1941.0, (28, 36): 378.0, (28, 37): 343.0, (28, 38): 1443.0, (28, 39): 1572.0, (28, 40): 1927.0, (28, 41): 1852.0, (28, 42): 1951.0, (28, 43): 1278.0, (28, 44): 2361.0, (28, 45): 681.0, (28, 46): 336.0, (28, 47): 973.0, (28, 48): 862.0, (28, 49): 734.0, (28, 50): 879.0, (28, 51): 1176.0, (28, 52): 535.0, (28, 53): 1578.0, (28, 54): 685.0, (28, 55): 879.0, (28, 56): 705.0, (28, 57): 816.0, (28, 58): 638.0, (28, 59): 1749.0, (28, 60): 1137.0, (28, 61): 1889.0, (28, 62): 935.0, (28, 63): 1127.0, (28, 64): 1989.0, (28, 65): 1138.0, (28, 66): 851.0, (28, 67): 909.0, (28, 68): 683.0, (28, 69): 479.0, (28, 70): 868.0, (28, 71): 1148.0, (28, 72): 597.0, (28, 73): 766.0, (28, 74): 215.0, (28, 75): 249.0, (28, 76): 1145.0, (28, 77): 1976.0, (28, 78): 1044.0, (28, 79): 790.0, (28, 80): 3891.0, (28, 82): 2350.0, (28, 83): 3085.0, (28, 84): 433.0, (28, 85): 1613.0, (28, 86): 1156.0, (28, 87): 293.0, (28, 88): 530.0, (28, 89): 518.0, (28, 90): 2001.0, (28, 91): 620.0, (28, 92): 905.0, (28, 93): 942.0, (28, 94): 1149.0, (28, 95): 1865.0, (28, 96): 849.0, (28, 97): 2149.0, (28, 98): 1805.0, (28, 99): 1738.0, (28, 100): 910.0, (28, 101): 1256.0, (28, 102): 806.0, (28, 103): 2189.0, (28, 104): 770.0, (28, 105): 1840.0, (28, 106): 1005.0, (28, 107): 253.0, (28, 108): 2307.0, (28, 109): 464.0, (28, 110): 943.0, (28, 111): 1448.0, (28, 112): 1519.0, (28, 113): 689.0, (28, 114): 338.0, (28, 115): 207.0, (28, 116): 956.0, (28, 117): 1073.0, (28, 118): 1973.0, (28, 119): 1781.0, (28, 121): 1043.0, (28, 122): 855.0, (28, 123): 1210.0, (28, 124): 1791.0, (28, 125): 1742.0, (28, 126): 996.0, (28, 127): 1642.0, (28, 128): 1285.0, (28, 129): 1969.0, (28, 130): 354.0, (28, 131): 1956.0, (28, 132): 2994.0, (28, 133): 631.0, (28, 134): 1957.0, (28, 136): 1206.0, (28, 137): 523.0, (28, 138): 629.0, (28, 139): 960.0, (28, 140): 1098.0, (28, 141): 852.0, (28, 142): 500.0, (28, 143): 801.0, (28, 144): 732.0, (28, 145): 344.0, (28, 146): 308.0, (28, 147): 385.0, (28, 148): 418.0, (28, 149): 525.0, (28, 150): 953.0, (28, 151): 2872.0, (28, 152): 331.0, (28, 153): 1215.0, (28, 154): 1145.0, (28, 155): 892.0, (28, 156): 825.0, (28, 157): 1973.0, (28, 158): 1642.0, (28, 159): 1248.0, (28, 160): 1152.0, (28, 161): 1983.0, (28, 162): 1472.0, (28, 163): 1259.0, (28, 164): 1973.0, (28, 165): 660.0, (28, 166): 1761.0, (28, 167): 528.0, (28, 168): 797.0, (28, 169): 1019.0, (28, 170): 595.0, (28, 171): 886.0, (28, 172): 652.0, (28, 173): 630.0, (28, 174): 2108.0, (28, 175): 713.0, (28, 176): 752.0, (28, 177): 794.0, (28, 178): 156.0, (28, 179): 1313.0, (28, 180): 1329.0, (28, 181): 1151.0, (28, 182): 202.0, (28, 183): 3370.0, (28, 184): 1969.0, (28, 185): 1318.0, (28, 186): 393.0, (28, 187): 611.0, (28, 188): 1643.0, (28, 189): 2243.0, (28, 190): 1966.0, (28, 191): 847.0, (28, 192): 771.0, (28, 193): 1984.0, (28, 194): 859.0, (28, 195): 2002.0, (28, 196): 950.0, (28, 197): 1974.0, (28, 198): 1461.0, (28, 199): 1872.0, (28, 200): 1846.0, (40, 1): 3661.0, (40, 2): 902.0, (40, 3): 2195.0, (40, 4): 817.0, (40, 5): 1444.0, (40, 6): 100.0, (40, 7): 2645.0, (40, 8): 1763.0, (40, 9): 1384.0, (40, 10): 1965.0, (40, 11): 464.0, (40, 12): 1163.0, (40, 13): 2247.0, (40, 14): 1724.0, (40, 15): 2557.0, (40, 16): 675.0, (40, 17): 565.0, (40, 18): 1352.0, (40, 19): 1168.0, (40, 20): 2154.0, (40, 21): 351.0, (40, 22): 2030.0, (40, 23): 1236.0, (40, 24): 2192.0, (40, 25): 977.0, (40, 26): 1134.0, (40, 27): 119.0, (40, 28): 1927.0, (40, 29): 2160.0, (40, 30): 1885.0, (40, 31): 1619.0, (40, 32): 2379.0, (40, 33): 50.0, (40, 34): 1675.0, (40, 35): 17.0, (40, 36): 2305.0, (40, 37): 2239.0, (40, 38): 694.0, (40, 39): 357.0, (40, 40): 0.0, (40, 41): 385.0, (40, 42): 762.0, (40, 43): 1936.0, (40, 44): 456.0, (40, 45): 1500.0, (40, 46): 1840.0, (40, 47): 1048.0, (40, 48): 2789.0, (40, 49): 1429.0, (40, 50): 1048.0, (40, 51): 3097.0, (40, 52): 1639.0, (40, 53): 424.0, (40, 54): 1362.0, (40, 55): 1972.0, (40, 56): 1251.0, (40, 57): 1118.0, (40, 58): 1292.0, (40, 59): 387.0, (40, 60): 810.0, (40, 61): 118.0, (40, 62): 2831.0, (40, 63): 3045.0, (40, 64): 62.0, (40, 65): 1026.0, (40, 66): 1348.0, (40, 67): 1128.0, (40, 68): 2605.0, (40, 69): 1548.0, (40, 70): 1376.0, (40, 71): 788.0, (40, 72): 1401.0, (40, 73): 1991.0, (40, 74): 2083.0, (40, 75): 2130.0, (40, 76): 783.0, (40, 77): 130.0, (40, 78): 2939.0, (40, 79): 1308.0, (40, 80): 3941.0, (40, 82): 1091.0, (40, 83): 3499.0, (40, 84): 1851.0, (40, 85): 1706.0, (40, 86): 1042.0, (40, 87): 2220.0, (40, 88): 2455.0, (40, 89): 1417.0, (40, 90): 427.0, (40, 91): 1645.0, (40, 92): 1164.0, (40, 93): 1287.0, (40, 94): 1014.0, (40, 95): 1013.0, (40, 96): 1086.0, (40, 97): 278.0, (40, 98): 341.0, (40, 99): 1249.0, (40, 100): 2810.0, (40, 101): 671.0, (40, 102): 1131.0, (40, 103): 505.0, (40, 104): 1238.0, (40, 105): 1086.0, (40, 106): 1115.0, (40, 107): 2168.0, (40, 108): 1228.0, (40, 109): 1891.0, (40, 110): 1179.0, (40, 111): 504.0, (40, 112): 638.0, (40, 113): 1408.0, (40, 114): 1592.0, (40, 115): 1961.0, (40, 116): 2857.0, (40, 117): 2977.0, (40, 118): 121.0, (40, 119): 319.0, (40, 121): 1318.0, (40, 122): 1212.0, (40, 123): 759.0, (40, 124): 350.0, (40, 125): 190.0, (40, 126): 1055.0, (40, 127): 478.0, (40, 128): 806.0, (40, 129): 127.0, (40, 130): 2281.0, (40, 131): 111.0, (40, 132): 3415.0, (40, 133): 1663.0, (40, 134): 111.0, (40, 136): 1078.0, (40, 137): 2449.0, (40, 138): 1300.0, (40, 139): 2887.0, (40, 140): 955.0, (40, 141): 1164.0, (40, 142): 1500.0, (40, 143): 1640.0, (40, 144): 1229.0, (40, 145): 1813.0, (40, 146): 1891.0, (40, 147): 1818.0, (40, 148): 2345.0, (40, 149): 2423.0, (40, 150): 978.0, (40, 151): 3282.0, (40, 152): 1612.0, (40, 153): 920.0, (40, 154): 835.0, (40, 155): 1045.0, (40, 156): 1169.0, (40, 157): 201.0, (40, 158): 513.0, (40, 159): 1309.0, (40, 160): 995.0, (40, 161): 131.0, (40, 162): 527.0, (40, 163): 1326.0, (40, 164): 135.0, (40, 165): 1555.0, (40, 166): 1860.0, (40, 167): 1399.0, (40, 168): 1357.0, (40, 169): 1004.0, (40, 170): 1949.0, (40, 171): 1212.0, (40, 172): 1296.0, (40, 173): 1324.0, (40, 174): 193.0, (40, 175): 1639.0, (40, 176): 1196.0, (40, 177): 2658.0, (40, 178): 1780.0, (40, 179): 865.0, (40, 180): 1828.0, (40, 181): 869.0, (40, 182): 2125.0, (40, 183): 3711.0, (40, 184): 109.0, (40, 185): 696.0, (40, 186): 1588.0, (40, 187): 1974.0, (40, 188): 475.0, (40, 189): 364.0, (40, 190): 82.0, (40, 191): 1293.0, (40, 192): 1179.0, (40, 193): 66.0, (40, 194): 1177.0, (40, 195): 99.0, (40, 196): 1097.0, (40, 197): 109.0, (40, 198): 523.0, (40, 199): 1094.0, (40, 200): 814.0,
                (47, 1): 3763.0, (47, 2): 300.0, (47, 3): 1233.0, (47, 4): 1462.0, (47, 5): 844.0, (47, 6): 1104.0, (47, 7): 1723.0, (47, 8): 1049.0, (47, 9): 1305.0, (47, 10): 1468.0, (47, 11): 1473.0, (47, 12): 403.0, (47, 13): 1272.0, (47, 14): 946.0, (47, 15): 1581.0, (47, 16): 831.0, (47, 17): 734.0, (47, 18): 961.0, (47, 19): 171.0, (47, 20): 1256.0, (47, 21): 810.0, (47, 22): 1207.0, (47, 23): 1541.0, (47, 24): 1340.0, (47, 25): 1310.0, (47, 26): 289.0, (47, 27): 1053.0, (47, 28): 973.0, (47, 29): 1163.0, (47, 30): 1257.0, (47, 31): 876.0, (47, 32): 1376.0, (47, 33): 1024.0, (47, 34): 820.0, (47, 35): 1064.0, (47, 36): 1333.0, (47, 37): 1313.0, (47, 38): 471.0, (47, 39): 701.0, (47, 40): 1048.0, (47, 41): 1118.0, (47, 42): 1375.0, (47, 43): 1602.0, (47, 44): 1504.0, (47, 45): 879.0, (47, 46): 1021.0, (47, 47): 0.0, (47, 48): 1812.0, (47, 49): 841.0, (47, 50): 328.0, (47, 51): 2096.0, (47, 52): 926.0, (47, 53): 826.0, (47, 54): 707.0, (47, 55): 1403.0, (47, 56): 522.0, (47, 57): 402.0, (47, 58): 385.0, (47, 59): 795.0, (47, 60): 487.0, (47, 61): 981.0, (47, 62): 1907.0, (47, 63): 2040.0, (47, 64): 1106.0, (47, 65): 214.0, (47, 66): 854.0, (47, 67): 80.0, (47, 68): 1612.0, (47, 69): 771.0, (47, 70): 899.0, (47, 71): 308.0, (47, 72): 664.0, (47, 73): 1358.0, (47, 74): 1168.0, (47, 75): 1211.0, (47, 76): 407.0, (47, 77): 1064.0, (47, 78): 1918.0, (47, 79): 747.0, (47, 80): 4115.0, (47, 82): 1812.0, (47, 83): 3456.0, (47, 84): 1078.0, (47, 85): 1661.0, (47, 86): 842.0, (47, 87): 1255.0, (47, 88): 1473.0, (47, 89): 564.0, (47, 90): 1265.0, (47, 91): 988.0, (47, 92): 686.0, (47, 93): 873.0, (47, 94): 219.0, (47, 95): 1436.0, (47, 96): 230.0, (47, 97): 1310.0, (47, 98): 854.0, (47, 99): 1474.0, (47, 100): 1794.0, (47, 101): 450.0, (47, 102): 240.0, (47, 103): 1437.0, (47, 104): 206.0, (47, 105): 1457.0, (47, 106): 139.0, (47, 107): 1226.0, (47, 108): 1850.0, (47, 109): 1129.0, (47, 110): 754.0, (47, 111): 680.0, (47, 112): 547.0, (47, 113): 775.0, (47, 114): 647.0, (47, 115): 1077.0, (47, 116): 1928.0, (47, 117): 1960.0, (47, 118): 1063.0, (47, 119): 836.0, (47, 121): 991.0, (47, 122): 165.0, (47, 123): 291.0, (47, 124): 839.0, (47, 125): 888.0, (47, 126): 47.0, (47, 127): 946.0, (47, 128): 762.0, (47, 129): 1058.0, (47, 130): 1316.0, (47, 131): 1047.0, (47, 132): 3365.0, (47, 133): 1012.0, (47, 134): 1049.0, (47, 136): 927.0, (47, 137): 1482.0, (47, 138): 448.0, (47, 139): 1912.0, (47, 140): 669.0, (47, 141): 124.0, (47, 142): 719.0, (47, 143): 1099.0, (47, 144): 250.0, (47, 145): 999.0, (47, 146): 1057.0, (47, 147): 1023.0, (47, 148): 1378.0, (47, 149): 1497.0, (47, 150): 241.0, (47, 151): 3230.0, (47, 152): 726.0, (47, 153): 252.0, (47, 154): 566.0, (47, 155): 211.0, (47, 156): 148.0, (47, 157): 1155.0, (47, 158): 677.0, (47, 159): 1148.0, (47, 160): 789.0, (47, 161): 1071.0, (47, 162): 755.0, (47, 163): 1169.0, (47, 164): 1059.0, (47, 165): 922.0, (47, 166): 1843.0, (47, 167): 492.0, (47, 168): 814.0, (47, 169): 616.0, (47, 170): 1240.0, (47, 171): 729.0, (47, 172): 528.0, (47, 173): 349.0, (47, 174): 1241.0, (47, 175): 1041.0, (47, 176): 249.0, (47, 177): 1630.0, (47, 178): 817.0, (47, 179): 853.0, (47, 180): 1562.0, (47, 181): 636.0, (47, 182): 1173.0, (47, 183): 3718.0, (47, 184): 1062.0, (47, 185): 688.0, (47, 186): 754.0, (47, 187): 1267.0, (47, 188): 683.0, (47, 189): 1403.0, (47, 190): 1107.0, (47, 191): 788.0, (47, 192): 230.0, (47, 193): 1093.0, (47, 194): 644.0, (47, 195): 1102.0, (47, 196): 653.0, (47, 197): 1067.0, (47, 198): 733.0, (47, 199): 1485.0, (47, 200): 1318.0, (48, 1): 3725.0, (48, 2): 1888.0, (48, 3): 595.0, (48, 4): 2840.0, (48, 5): 1487.0, (48, 6): 2865.0, (48, 7): 298.0, (48, 8): 1193.0, (48, 9): 2075.0, (48, 10): 1458.0, (48, 11): 3132.0, (48, 12): 1630.0, (48, 13): 544.0, (48, 14): 1153.0, (48, 15): 233.0, (48, 16): 2272.0, (48, 17): 2290.0, (48, 18): 1714.0, (48, 19): 1655.0, (48, 20): 684.0, (48, 21): 2613.0, (48, 22): 882.0, (48, 23): 2571.0, (48, 24): 726.0, (48, 25): 2506.0, (48, 26): 1657.0, (48, 27): 2825.0, (48, 28): 862.0, (48, 29): 651.0, (48, 30): 1253.0, (48, 31): 1259.0, (48, 32): 450.0, (48, 33): 2778.0, (48, 34): 1137.0, (48, 35): 2804.0, (48, 36): 485.0, (48, 37): 577.0, (48, 38): 2281.0, (48, 39): 2434.0, (48, 40): 2789.0, (48, 41): 2702.0, (48, 42): 2765.0, (48, 43): 1750.0, (48, 44): 3222.0, (48, 45): 1437.0, (48, 46): 1025.0, (48, 47): 1812.0, (48, 48): 0.0, (48, 49): 1506.0, (48, 50): 1742.0, (48, 51): 338.0, (48, 52): 1271.0, (48, 53): 2435.0, (48, 54): 1501.0, (48, 55): 1329.0, (48, 56): 1557.0, (48, 57): 1676.0, (48, 58): 1499.0, (48, 59): 2601.0, (48, 60): 1995.0, (48, 61): 2750.0, (48, 62): 284.0, (48, 63): 308.0, (48, 64): 2852.0, (48, 65): 1951.0, (48, 66): 1623.0, (48, 67): 1741.0, (48, 68): 211.0, (48, 69): 1290.0, (48, 70): 1625.0, (48, 71): 2009.0, (48, 72): 1429.0, (48, 73): 1211.0, (48, 74): 728.0, (48, 75): 682.0, (48, 76): 2008.0, (48, 77): 2838.0, (48, 78): 340.0, (48, 79): 1594.0, (48, 80): 4132.0, (48, 82): 3137.0, (48, 83): 3241.0, (48, 84): 1068.0, (48, 85): 2198.0, (48, 86): 1962.0, (48, 87): 570.0, (48, 88): 339.0, (48, 89): 1377.0, (48, 90): 2849.0, (48, 91): 1321.0, (48, 92): 1727.0, (48, 93): 1716.0, (48, 94): 1963.0, (48, 95): 2638.0, (48, 96): 1709.0, (48, 97): 3009.0, (48, 98): 2658.0, (48, 99): 2461.0, (48, 100): 265.0, (48, 101): 2118.0, (48, 102): 1665.0, (48, 103): 3037.0, (48, 104): 1606.0, (48, 105): 2599.0, (48, 106): 1818.0, (48, 107): 629.0, (48, 108): 3067.0, (48, 109): 1055.0, (48, 110): 1752.0, (48, 111): 2308.0, (48, 112): 2358.0, (48, 113): 1484.0, (48, 114): 1199.0, (48, 115): 863.0, (48, 116): 269.0, (48, 117): 326.0, (48, 118): 2835.0, (48, 119): 2636.0, (48, 121): 1787.0, (48, 122): 1674.0, (48, 123): 2065.0, (48, 124): 2644.0, (48, 125): 2604.0, (48, 126): 1828.0, (48, 127): 2490.0, (48, 128): 2122.0, (48, 129): 2831.0, (48, 130): 508.0, (48, 131): 2817.0, (48, 132): 3160.0, (48, 133): 1318.0, (48, 134): 2819.0, (48, 136): 1996.0, (48, 137): 341.0, (48, 138): 1491.0, (48, 139): 101.0, (48, 140): 1934.0, (48, 141): 1688.0, (48, 142): 1327.0, (48, 143): 1456.0, (48, 144): 1583.0, (48, 145): 1050.0, (48, 146): 969.0, (48, 147): 1067.0, (48, 148): 445.0, (48, 149): 419.0, (48, 150): 1814.0, (48, 151): 3059.0, (48, 152): 1184.0, (48, 153): 2039.0, (48, 154): 1997.0, (48, 155): 1752.0, (48, 156): 1667.0, (48, 157): 2833.0, (48, 158): 2488.0, (48, 159): 1969.0, (48, 160): 1970.0, (48, 161): 2844.0, (48, 162): 2327.0, (48, 163): 1974.0, (48, 164): 2834.0, (48, 165): 1396.0, (48, 166): 2302.0, (48, 167): 1391.0, (48, 168): 1580.0, (48, 169): 1858.0, (48, 170): 1100.0, (48, 171): 1697.0, (48, 172): 1506.0, (48, 173): 1482.0, (48, 174): 2970.0, (48, 175): 1390.0, (48, 176): 1607.0, (48, 177): 392.0, (48, 178): 1012.0, (48, 179): 2136.0, (48, 180): 1854.0, (48, 181): 1996.0, (48, 182): 666.0, (48, 183): 3528.0, (48, 184): 2831.0, (48, 185): 2169.0, (48, 186): 1225.0, (48, 187): 1093.0, (48, 188): 2492.0, (48, 189): 3103.0, (48, 190): 2828.0, (48, 191): 1641.0, (48, 192): 1626.0, (48, 193): 2846.0, (48, 194): 1689.0, (48, 195): 2864.0, (48, 196): 1781.0, (48, 197): 2835.0, (48, 198): 2318.0, (48, 199): 2631.0, (48, 200): 2652.0,
                (68, 1): 3755.0, (68, 2): 1702.0, (68, 3): 423.0, (68, 4): 2695.0, (68, 5): 1348.0, (68, 6): 2678.0, (68, 7): 354.0, (68, 8): 1075.0, (68, 9): 1968.0, (68, 10): 1402.0, (68, 11): 2960.0, (68, 12): 1451.0, (68, 13): 362.0, (68, 14): 1016.0, (68, 15): 96.0, (68, 16): 2109.0, (68, 17): 2118.0, (68, 18): 1586.0, (68, 19): 1459.0, (68, 20): 562.0, (68, 21): 2418.0, (68, 22): 776.0, (68, 23): 2456.0, (68, 24): 641.0, (68, 25): 2372.0, (68, 26): 1471.0, (68, 27): 2635.0, (68, 28): 683.0, (68, 29): 450.0, (68, 30): 1170.0, (68, 31): 1118.0, (68, 32): 241.0, (68, 33): 2592.0, (68, 34): 977.0, (68, 35): 2619.0, (68, 36): 309.0, (68, 37): 448.0, (68, 38): 2080.0, (68, 39): 2248.0, (68, 40): 2605.0, (68, 41): 2534.0, (68, 42): 2617.0, (68, 43): 1697.0, (68, 44): 3043.0, (68, 45): 1302.0, (68, 46): 891.0, (68, 47): 1612.0, (68, 48): 211.0, (68, 49): 1367.0, (68, 50): 1560.0, (68, 51): 494.0, (68, 52): 1137.0, (68, 53): 2261.0, (68, 54): 1347.0, (68, 55): 1268.0, (68, 56): 1386.0, (68, 57): 1498.0, (68, 58): 1313.0, (68, 59): 2404.0, (68, 60): 1820.0, (68, 61): 2561.0, (68, 62): 438.0, (68, 63): 445.0, (68, 64): 2666.0, (68, 65): 1746.0, (68, 66): 1486.0, (68, 67): 1540.0, (68, 68): 0.0, (68, 69): 1135.0, (68, 70): 1492.0, (68, 71): 1820.0, (68, 72): 1269.0, (68, 73): 1145.0, (68, 74): 583.0, (68, 75): 540.0, (68, 76): 1826.0, (68, 77): 2648.0, (68, 78): 385.0, (68, 79): 1446.0, (68, 80): 4161.0, (68, 82): 3002.0, (68, 83): 3286.0, (68, 84): 949.0, (68, 85): 2123.0, (68, 86): 1815.0, (68, 87): 397.0, (68, 88): 156.0, (68, 89): 1201.0, (68, 90): 2683.0, (68, 91): 1198.0, (68, 92): 1572.0, (68, 93): 1579.0, (68, 94): 1758.0, (68, 95): 2507.0, (68, 96): 1520.0, (68, 97): 2831.0, (68, 98): 2462.0, (68, 99): 2347.0, (68, 100): 256.0, (68, 101): 1934.0, (68, 102): 1476.0, (68, 103): 2871.0, (68, 104): 1406.0, (68, 105): 2473.0, (68, 106): 1614.0, (68, 107): 473.0, (68, 108): 2942.0, (68, 109): 945.0, (68, 110): 1603.0, (68, 111): 2131.0, (68, 112): 2157.0, (68, 113): 1338.0, (68, 114): 1014.0, (68, 115): 720.0, (68, 116): 436.0, (68, 117): 402.0, (68, 118): 2646.0, (68, 119): 2442.0, (68, 121): 1660.0, (68, 122): 1471.0, (68, 123): 1872.0, (68, 124): 2448.0, (68, 125): 2422.0, (68, 126): 1627.0, (68, 127): 2323.0, (68, 128): 1962.0, (68, 129): 2641.0, (68, 130): 340.0, (68, 131): 2628.0, (68, 132): 3202.0, (68, 133): 1199.0, (68, 134): 2630.0, (68, 136): 1856.0, (68, 137): 194.0, (68, 138): 1310.0, (68, 139): 311.0, (68, 140): 1773.0, (68, 141): 1488.0, (68, 142): 1167.0, (68, 143): 1349.0, (68, 144): 1388.0, (68, 145): 914.0, (68, 146): 837.0, (68, 147): 938.0, (68, 148): 281.0, (68, 149): 345.0, (68, 150): 1627.0, (68, 151): 3095.0, (68, 152): 1012.0, (68, 153): 1836.0, (68, 154): 1827.0, (68, 155): 1563.0, (68, 156): 1469.0, (68, 157): 2656.0, (68, 158): 2289.0, (68, 159): 1852.0, (68, 160): 1818.0, (68, 161): 2655.0, (68, 162): 2155.0, (68, 163): 1859.0, (68, 164): 2644.0, (68, 165): 1266.0, (68, 166): 2240.0, (68, 167): 1207.0, (68, 168): 1438.0, (68, 169): 1696.0, (68, 170): 1013.0, (68, 171): 1547.0, (68, 172): 1335.0, (68, 173): 1288.0, (68, 174): 2788.0, (68, 175): 1275.0, (68, 176): 1415.0, (68, 177): 263.0, (68, 178): 825.0, (68, 179): 1983.0, (68, 180): 1789.0, (68, 181): 1831.0, (68, 182): 498.0, (68, 183): 3574.0, (68, 184): 2642.0, (68, 185): 2000.0, (68, 186): 1062.0, (68, 187): 1011.0, (68, 188): 2294.0, (68, 189): 2925.0, (68, 190): 2647.0, (68, 191): 1496.0, (68, 192): 1433.0, (68, 193): 2660.0, (68, 194): 1531.0, (68, 195): 2677.0, (68, 196): 1623.0, (68, 197): 2647.0, (68, 198): 2144.0, (68, 199): 2505.0, (68, 200): 2508.0, (69, 1): 3192.0, (69, 2): 686.0, (69, 3): 713.0, (69, 4): 1560.0, (69, 5): 246.0, (69, 6): 1634.0, (69, 7): 1107.0, (69, 8): 279.0, (69, 9): 926.0, (69, 10): 744.0, (69, 11): 1852.0, (69, 12): 414.0, (69, 13): 779.0, (69, 14): 184.0, (69, 15): 1067.0, (69, 16): 983.0, (69, 17): 1014.0, (69, 18): 499.0, (69, 19): 608.0, (69, 20): 617.0, (69, 21): 1445.0, (69, 22): 483.0, (69, 23): 1367.0, (69, 24): 644.0, (69, 25): 1249.0, (69, 26): 500.0, (69, 27): 1603.0, (69, 28): 479.0, (69, 29): 750.0, (69, 30): 497.0, (69, 31): 107.0, (69, 32): 952.0, (69, 33): 1546.0, (69, 34): 163.0, (69, 35): 1561.0, (69, 36): 827.0, (69, 37): 715.0, (69, 38): 1197.0, (69, 39): 1207.0, (69, 40): 1548.0, (69, 41): 1417.0, (69, 42): 1482.0, (69, 43): 950.0, (69, 44): 1957.0, (69, 45): 225.0, (69, 46): 293.0, (69, 47): 771.0, (69, 48): 1290.0, (69, 49): 261.0, (69, 50): 536.0, (69, 51): 1620.0, (69, 52): 164.0, (69, 53): 1162.0, (69, 54): 212.0, (69, 55): 654.0, (69, 56): 303.0, (69, 57): 449.0, (69, 58): 390.0, (69, 59): 1443.0, (69, 60): 738.0, (69, 61): 1530.0, (69, 62): 1291.0, (69, 63): 1577.0, (69, 64): 1610.0, (69, 65): 980.0, (69, 66): 375.0, (69, 67): 741.0, (69, 68): 1135.0, (69, 69): 0.0, (69, 70): 396.0, (69, 71): 822.0, (69, 72): 147.0, (69, 73): 592.0, (69, 74): 562.0, (69, 75): 608.0, (69, 76): 777.0, (69, 77): 1618.0, (69, 78): 1510.0, (69, 79): 313.0, (69, 80): 3571.0, (69, 82): 1874.0, (69, 83): 2824.0, (69, 84): 318.0, (69, 85): 1193.0, (69, 86): 682.0, (69, 87): 739.0, (69, 88): 979.0, (69, 89): 213.0, (69, 90): 1563.0, (69, 91): 247.0, (69, 92): 438.0, (69, 93): 465.0, (69, 94): 988.0, (69, 95): 1387.0, (69, 96): 565.0, (69, 97): 1742.0, (69, 98): 1491.0, (69, 99): 1264.0, (69, 100): 1376.0, (69, 101): 894.0, (69, 102): 539.0, (69, 103): 1750.0, (69, 104): 624.0, (69, 105): 1361.0, (69, 106): 865.0, (69, 107): 664.0, (69, 108): 1828.0, (69, 109): 366.0, (69, 110): 469.0, (69, 111): 1048.0, (69, 112): 1264.0, (69, 113): 211.0, (69, 114): 304.0, (69, 115): 428.0, (69, 116): 1319.0, (69, 117): 1535.0, (69, 118): 1613.0, (69, 119): 1460.0, (69, 121): 572.0, (69, 122): 731.0, (69, 123): 916.0, (69, 124): 1478.0, (69, 125): 1358.0, (69, 126): 812.0, (69, 127): 1205.0, (69, 128): 832.0, (69, 129): 1611.0, (69, 130): 795.0, (69, 131): 1595.0, (69, 132): 2732.0, (69, 133): 270.0, (69, 134): 1596.0, (69, 136): 728.0, (69, 137): 954.0, (69, 138): 324.0, (69, 139): 1380.0, (69, 140): 645.0, (69, 141): 683.0, (69, 142): 53.0, (69, 143): 415.0, (69, 144): 535.0, (69, 145): 265.0, (69, 146): 344.0, (69, 147): 274.0, (69, 148): 856.0, (69, 149): 892.0, (69, 150): 640.0, (69, 151): 2602.0, (69, 152): 187.0, (69, 153): 1020.0, (69, 154): 722.0, (69, 155): 604.0, (69, 156): 642.0, (69, 157): 1563.0, (69, 158): 1362.0, (69, 159): 780.0, (69, 160): 683.0, (69, 161): 1624.0, (69, 162): 1053.0, (69, 163): 793.0, (69, 164): 1615.0, (69, 165): 227.0, (69, 166): 1356.0, (69, 167): 302.0, (69, 168): 319.0, (69, 169): 571.0, (69, 170): 469.0, (69, 171): 412.0, (69, 172): 267.0, (69, 173): 462.0, (69, 174): 1717.0, (69, 175): 331.0, (69, 176): 524.0, (69, 177): 1270.0, (69, 178): 401.0, (69, 179): 848.0, (69, 180): 961.0, (69, 181): 711.0, (69, 182): 638.0, (69, 183): 3100.0, (69, 184): 1607.0, (69, 185): 888.0, (69, 186): 95.0, (69, 187): 496.0, (69, 188): 1352.0, (69, 189): 1834.0, (69, 190): 1575.0, (69, 191): 368.0, (69, 192): 542.0, (69, 193): 1610.0, (69, 194): 400.0, (69, 195): 1633.0, (69, 196): 491.0, (69, 197): 1611.0, (69, 198): 1046.0, (69, 199): 1394.0, (69, 200): 1374.0, (77, 1): 3790.0, (77, 2): 953.0, (77, 3): 2245.0, (77, 4): 942.0, (77, 5): 1529.0, (77, 6): 63.0, (77, 7): 2705.0, (77, 8): 1843.0, (77, 9): 1505.0, (77, 10): 2068.0, (77, 11): 531.0, (77, 12): 1221.0, (77, 13): 2294.0, (77, 14): 1796.0, (77, 15): 2605.0, (77, 16): 786.0, (77, 17): 664.0, (77, 18): 1452.0, (77, 19): 1197.0, (77, 20): 2215.0, (77, 21): 294.0, (77, 22): 2101.0, (77, 23): 1366.0, (77, 24): 2261.0, (77, 25): 1106.0, (77, 26): 1181.0, (77, 27): 14.0, (77, 28): 1976.0, (77, 29): 2201.0, (77, 30): 1975.0, (77, 31): 1693.0, (77, 32): 2419.0, (77, 33): 93.0, (77, 34): 1737.0, (77, 35): 133.0, (77, 36): 2353.0, (77, 37): 2296.0, (77, 38): 667.0, (77, 39): 411.0, (77, 40): 130.0, (77, 41): 514.0, (77, 42): 890.0, (77, 43): 2049.0, (77, 44): 472.0, (77, 45): 1585.0, (77, 46): 1910.0, (77, 47): 1064.0, (77, 48): 2838.0, (77, 49): 1514.0, (77, 50): 1103.0, (77, 51): 3139.0, (77, 52): 1717.0, (77, 53): 531.0, (77, 54): 1440.0, (77, 55): 2069.0, (77, 56): 1317.0, (77, 57): 1178.0, (77, 58): 1338.0, (77, 59): 327.0, (77, 60): 883.0, (77, 61): 88.0, (77, 62): 2892.0, (77, 63): 3086.0, (77, 64): 116.0, (77, 65): 1016.0, (77, 66): 1440.0, (77, 67): 1144.0, (77, 68): 2648.0, (77, 69): 1618.0, (77, 70): 1470.0, (77, 71): 829.0, (77, 72): 1472.0, (77, 73): 2083.0, (77, 74): 2141.0, (77, 75): 2187.0, (77, 76): 843.0, (77, 77): 0.0, (77, 78): 2974.0, (77, 79): 1392.0, (77, 80): 4069.0, (77, 82): 1205.0, (77, 83): 3628.0, (77, 84): 1926.0, (77, 85): 1831.0, (77, 86): 1146.0, (77, 87): 2269.0, (77, 88): 2501.0, (77, 89): 1474.0, (77, 90): 547.0, (77, 91): 1730.0, (77, 92): 1250.0, (77, 93): 1383.0, (77, 94): 1003.0, (77, 95): 1143.0, (77, 96): 1129.0, (77, 97): 343.0, (77, 98): 269.0, (77, 99): 1378.0, (77, 100): 2846.0, (77, 101): 727.0, (77, 102): 1173.0, (77, 103): 603.0, (77, 104): 1262.0, (77, 105): 1216.0, (77, 106): 1116.0, (77, 107): 2221.0, (77, 108): 1348.0, (77, 109): 1968.0, (77, 110): 1271.0, (77, 111): 590.0, (77, 112): 602.0, (77, 113): 1489.0, (77, 114): 1639.0, (77, 115): 2023.0, (77, 116): 2917.0, (77, 117): 3013.0, (77, 118): 8.0, (77, 119): 260.0, (77, 121): 1422.0, (77, 122): 1226.0, (77, 123): 782.0, (77, 124): 281.0, (77, 125): 282.0, (77, 126): 1065.0, (77, 127): 600.0, (77, 128): 910.0, (77, 129): 7.0, (77, 130): 2331.0, (77, 131): 24.0, (77, 132): 3544.0, (77, 133): 1748.0, (77, 134): 23.0, (77, 136): 1187.0, (77, 137): 2499.0, (77, 138): 1353.0, (77, 139): 2936.0, (77, 140): 1046.0, (77, 141): 1185.0, (77, 142): 1568.0, (77, 143): 1735.0, (77, 144): 1263.0, (77, 145): 1883.0, (77, 146): 1959.0, (77, 147): 1890.0, (77, 148): 2394.0, (77, 149): 2481.0, (77, 150): 1024.0, (77, 151): 3410.0, (77, 152): 1669.0, (77, 153): 908.0, (77, 154): 918.0, (77, 155): 1086.0, (77, 156): 1194.0, (77, 157): 319.0, (77, 158): 464.0, (77, 159): 1424.0, (77, 160): 1096.0, (77, 161): 6.0, (77, 162): 628.0, (77, 163): 1441.0, (77, 164): 8.0, (77, 165): 1639.0, (77, 166): 1987.0, (77, 167): 1449.0, (77, 168): 1445.0, (77, 169): 1090.0, (77, 170): 2033.0, (77, 171): 1300.0, (77, 172): 1360.0, (77, 173): 1361.0, (77, 174): 232.0, (77, 175): 1729.0, (77, 176): 1234.0, (77, 177): 2689.0, (77, 178): 1826.0, (77, 179): 974.0, (77, 180): 1944.0, (77, 181): 960.0, (77, 182): 2176.0, (77, 183): 3840.0, (77, 184): 20.0, (77, 185): 792.0, (77, 186): 1652.0, (77, 187): 2058.0, (77, 188): 433.0, (77, 189): 410.0, (77, 190): 194.0, (77, 191): 1382.0, (77, 192): 1216.0, (77, 193): 87.0, (77, 194): 1259.0, (77, 195): 62.0, (77, 196): 1183.0, (77, 197): 22.0, (77, 198): 621.0, (77, 199): 1224.0, (77, 200): 943.0,
                (101, 1): 3499.0, (101, 2): 232.0, (101, 3): 1524.0, (101, 4): 1017.0, (101, 5): 835.0, (101, 6): 749.0, (101, 7): 1978.0, (101, 8): 1133.0, (101, 9): 1041.0, (101, 10): 1427.0, (101, 11): 1051.0, (101, 12): 494.0, (101, 13): 1576.0, (101, 14): 1075.0, (101, 15): 1886.0, (101, 16): 411.0, (101, 17): 289.0, (101, 18): 824.0, (101, 19): 522.0, (101, 20): 1488.0, (101, 21): 562.0, (101, 22): 1377.0, (101, 23): 1161.0, (101, 24): 1535.0, (101, 25): 908.0, (101, 26): 464.0, (101, 27): 713.0, (101, 28): 1256.0, (101, 29): 1491.0, (101, 30): 1288.0, (101, 31): 975.0, (101, 32): 1710.0, (101, 33): 661.0, (101, 34): 1010.0, (101, 35): 686.0, (101, 36): 1634.0, (101, 37): 1570.0, (101, 38): 444.0, (101, 39): 318.0, (101, 40): 671.0, (101, 41): 671.0, (101, 42): 931.0, (101, 43): 1476.0, (101, 44): 1111.0, (101, 45): 887.0, (101, 46): 1185.0, (101, 47): 450.0, (101, 48): 2118.0, (101, 49): 823.0, (101, 50): 377.0, (101, 51): 2427.0, (101, 52): 1005.0, (101, 53): 377.0, (101, 54): 729.0, (101, 55): 1403.0, (101, 56): 591.0, (101, 57): 451.0, (101, 58): 621.0, (101, 59): 567.0, (101, 60): 187.0, (101, 61): 640.0, (101, 62): 2165.0, (101, 63): 2375.0, (101, 64): 733.0, (101, 65): 543.0, (101, 66): 772.0, (101, 67): 524.0, (101, 68): 1934.0, (101, 69): 894.0, (101, 70): 810.0, (101, 71): 142.0, (101, 72): 749.0, (101, 73): 1397.0, (101, 74): 1415.0, (101, 75): 1461.0, (101, 76): 117.0, (101, 77): 727.0, (101, 78): 2273.0, (101, 79): 701.0, (101, 80): 3829.0, (101, 82): 1366.0, (101, 83): 3244.0, (101, 84): 1207.0, (101, 85): 1406.0, (101, 86): 577.0, (101, 87): 1548.0, (101, 88): 1784.0, (101, 89): 748.0, (101, 90): 821.0, (101, 91): 1029.0, (101, 92): 571.0, (101, 93): 740.0, (101, 94): 536.0, (101, 95): 1022.0, (101, 96): 421.0, (101, 97): 904.0, (101, 98): 606.0, (101, 99): 1112.0, (101, 100): 2142.0, (101, 101): 0.0, (101, 102): 466.0, (101, 103): 998.0, (101, 104): 602.0, (101, 105): 1056.0, (101, 106): 565.0, (101, 107): 1497.0, (101, 108): 1416.0, (101, 109): 1251.0, (101, 110): 612.0, (101, 111): 231.0, (101, 112): 476.0, (101, 113): 785.0, (101, 114): 920.0, (101, 115): 1296.0, (101, 116): 2190.0, (101, 117): 2309.0, (101, 118): 723.0, (101, 119): 573.0, (101, 121): 820.0, (101, 122): 608.0, (101, 123): 205.0, (101, 124): 595.0, (101, 125): 488.0, (101, 126): 477.0, (101, 127): 496.0, (101, 128): 397.0, (101, 129): 720.0, (101, 130): 1610.0, (101, 131): 705.0, (101, 132): 3154.0, (101, 133): 1049.0, (101, 134): 707.0, (101, 136): 651.0, (101, 137): 1778.0, (101, 138): 629.0, (101, 139): 2216.0, (101, 140): 415.0, (101, 141): 541.0, (101, 142): 844.0, (101, 143): 1067.0, (101, 144): 572.0, (101, 145): 1158.0, (101, 146): 1234.0, (101, 147): 1167.0, (101, 148): 1673.0, (101, 149): 1755.0, (101, 150): 312.0, (101, 151): 3018.0, (101, 152): 943.0, (101, 153): 472.0, (101, 154): 262.0, (101, 155): 381.0, (101, 156): 533.0, (101, 157): 735.0, (101, 158): 518.0, (101, 159): 907.0, (101, 160): 517.0, (101, 161): 733.0, (101, 162): 306.0, (101, 163): 928.0, (101, 164): 724.0, (101, 165): 941.0, (101, 166): 1585.0, (101, 167): 728.0, (101, 168): 762.0, (101, 169): 421.0, (101, 170): 1327.0, (101, 171): 625.0, (101, 172): 633.0, (101, 173): 661.0, (101, 174): 854.0, (101, 175): 1043.0, (101, 176): 535.0, (101, 177): 1995.0, (101, 178): 1109.0, (101, 179): 497.0, (101, 180): 1401.0, (101, 181): 336.0, (101, 182): 1454.0, (101, 183): 3489.0, (101, 184): 718.0, (101, 185): 281.0, (101, 186): 925.0, (101, 187): 1353.0, (101, 188): 495.0, (101, 189): 999.0, (101, 190): 713.0, (101, 191): 706.0, (101, 192): 520.0, (101, 193): 729.0, (101, 194): 564.0, (101, 195): 748.0, (101, 196): 508.0, (101, 197): 722.0, (101, 198): 283.0, (101, 199): 1081.0, (101, 200): 884.0, (102, 1): 3551.0, (102, 2): 238.0, (102, 3): 1074.0, (102, 4): 1402.0, (102, 5): 605.0, (102, 6): 1203.0, (102, 7): 1547.0, (102, 8): 814.0, (102, 9): 1111.0, (102, 10): 1229.0, (102, 11): 1515.0, (102, 12): 169.0, (102, 13): 1122.0, (102, 14): 718.0, (102, 15): 1433.0, (102, 16): 758.0, (102, 17): 703.0, (102, 18): 739.0, (102, 19): 116.0, (102, 20): 1066.0, (102, 21): 954.0, (102, 22): 993.0, (102, 23): 1403.0, (102, 24): 1136.0, (102, 25): 1196.0, (102, 26): 53.0, (102, 27): 1160.0, (102, 28): 806.0, (102, 29): 1029.0, (102, 30): 1018.0, (102, 31): 641.0, (102, 32): 1248.0, (102, 33): 1117.0, (102, 34): 606.0, (102, 35): 1146.0, (102, 36): 1181.0, (102, 37): 1135.0, (102, 38): 665.0, (102, 39): 774.0, (102, 40): 1131.0, (102, 41): 1118.0, (102, 42): 1316.0, (102, 43): 1369.0, (102, 44): 1577.0, (102, 45): 639.0, (102, 46): 803.0, (102, 47): 240.0, (102, 48): 1665.0, (102, 49): 603.0, (102, 50): 143.0, (102, 51): 1967.0, (102, 52): 690.0, (102, 53): 827.0, (102, 54): 469.0, (102, 55): 1164.0, (102, 56): 283.0, (102, 57): 181.0, (102, 58): 172.0, (102, 59): 946.0, (102, 60): 405.0, (102, 61): 1086.0, (102, 62): 1734.0, (102, 63): 1914.0, (102, 64): 1193.0, (102, 65): 454.0, (102, 66): 625.0, (102, 67): 235.0, (102, 68): 1476.0, (102, 69): 539.0, (102, 70): 670.0, (102, 71): 345.0, (102, 72): 425.0, (102, 73): 1120.0, (102, 74): 984.0, (102, 75): 1029.0, (102, 76): 371.0, (102, 77): 1173.0, (102, 78): 1808.0, (102, 79): 515.0, (102, 80): 3909.0, (102, 82): 1749.0, (102, 83): 3232.0, (102, 84): 851.0, (102, 85): 1458.0, (102, 86): 670.0, (102, 87): 1098.0, (102, 88): 1328.0, (102, 89): 339.0, (102, 90): 1270.0, (102, 91): 749.0, (102, 92): 475.0, (102, 93): 654.0, (102, 94): 459.0, (102, 95): 1330.0, (102, 96): 45.0, (102, 97): 1370.0, (102, 98): 1000.0, (102, 99): 1324.0, (102, 100): 1678.0, (102, 101): 466.0, (102, 102): 0.0, (102, 103): 1453.0, (102, 104): 192.0, (102, 105): 1338.0, (102, 106): 357.0, (102, 107): 1055.0, (102, 108): 1762.0, (102, 109): 901.0, (102, 110): 544.0, (102, 111): 684.0, (102, 112): 736.0, (102, 113): 536.0, (102, 114): 469.0, (102, 115): 879.0, (102, 116): 1757.0, (102, 117): 1845.0, (102, 118): 1170.0, (102, 119): 975.0, (102, 121): 778.0, (102, 122): 270.0, (102, 123): 405.0, (102, 124): 986.0, (102, 125): 952.0, (102, 126): 284.0, (102, 127): 923.0, (102, 128): 652.0, (102, 129): 1166.0, (102, 130): 1160.0, (102, 131): 1153.0, (102, 132): 3140.0, (102, 133): 773.0, (102, 134): 1154.0, (102, 136): 753.0, (102, 137): 1328.0, (102, 138): 216.0, (102, 139): 1764.0, (102, 140): 513.0, (102, 141): 193.0, (102, 142): 486.0, (102, 143): 860.0, (102, 144): 116.0, (102, 145): 779.0, (102, 146): 843.0, (102, 147): 800.0, (102, 148): 1223.0, (102, 149): 1321.0, (102, 150): 155.0, (102, 151): 3006.0, (102, 152): 520.0, (102, 153): 483.0, (102, 154): 458.0, (102, 155): 87.0, (102, 156): 153.0, (102, 157): 1201.0, (102, 158): 847.0, (102, 159): 951.0, (102, 160): 626.0, (102, 161): 1179.0, (102, 162): 733.0, (102, 163): 971.0, (102, 164): 1169.0, (102, 165): 682.0, (102, 166): 1639.0, (102, 167): 286.0, (102, 168): 581.0, (102, 169): 443.0, (102, 170): 1007.0, (102, 171): 511.0, (102, 172): 288.0, (102, 173): 195.0, (102, 174): 1318.0, (102, 175): 802.0, (102, 176): 72.0, (102, 177): 1529.0, (102, 178): 654.0, (102, 179): 727.0, (102, 180): 1335.0, (102, 181): 509.0, (102, 182): 1007.0, (102, 183): 3496.0, (102, 184): 1166.0, (102, 185): 618.0, (102, 186): 532.0, (102, 187): 1034.0, (102, 188): 844.0, (102, 189): 1465.0, (102, 190): 1178.0, (102, 191): 561.0, (102, 192): 64.0, (102, 193): 1185.0, (102, 194): 426.0, (102, 195): 1201.0, (102, 196): 455.0, (102, 197): 1171.0, (102, 198): 717.0, (102, 199): 1368.0, (102, 200): 1240.0, (109, 1): 3034.0, (109, 2): 1049.0, (109, 3): 569.0, (109, 4): 1803.0, (109, 5): 464.0, (109, 6): 1980.0, (109, 7): 820.0, (109, 8): 142.0, (109, 9): 1024.0, (109, 10): 543.0, (109, 11): 2163.0, (109, 12): 780.0, (109, 13): 639.0, (109, 14): 183.0, (109, 15): 859.0, (109, 16): 1285.0, (109, 17): 1341.0, (109, 18): 664.0, (109, 19): 961.0, (109, 20): 383.0, (109, 21): 1807.0, (109, 22): 173.0, (109, 23): 1517.0, (109, 24): 336.0, (109, 25): 1457.0, (109, 26): 865.0, (109, 27): 1954.0, (109, 28): 464.0, (109, 29): 681.0, (109, 30): 264.0, (109, 31): 276.0, (109, 32): 825.0, (109, 33): 1892.0, (109, 34): 327.0, (109, 35): 1903.0, (109, 36): 664.0, (109, 37): 497.0, (109, 38): 1562.0, (109, 39): 1559.0, (109, 40): 1891.0, (109, 41): 1724.0, (109, 42): 1732.0, (109, 43): 820.0, (109, 44): 2284.0, (109, 45): 408.0, (109, 46): 141.0, (109, 47): 1129.0, (109, 48): 1055.0, (109, 49): 482.0, (109, 50): 901.0, (109, 51): 1393.0, (109, 52): 253.0, (109, 53): 1490.0, (109, 54): 529.0, (109, 55): 415.0, (109, 56): 665.0, (109, 57): 813.0, (109, 58): 743.0, (109, 59): 1806.0, (109, 60): 1085.0, (109, 61): 1882.0, (109, 62): 995.0, (109, 63): 1360.0, (109, 64): 1953.0, (109, 65): 1333.0, (109, 66): 590.0, (109, 67): 1091.0, (109, 68): 945.0, (109, 69): 366.0, (109, 70): 584.0, (109, 71): 1187.0, (109, 72): 504.0, (109, 73): 306.0, (109, 74): 386.0, (109, 75): 418.0, (109, 76): 1134.0, (109, 77): 1968.0, (109, 78): 1330.0, (109, 79): 593.0, (109, 80): 3426.0, (109, 82): 2088.0, (109, 83): 2625.0, (109, 84): 51.0, (109, 85): 1193.0, (109, 86): 939.0, (109, 87): 591.0, (109, 88): 798.0, (109, 89): 565.0, (109, 90): 1863.0, (109, 91): 273.0, (109, 92): 737.0, (109, 93): 679.0, (109, 94): 1341.0, (109, 95): 1587.0, (109, 96): 930.0, (109, 97): 2068.0, (109, 98): 1852.0, (109, 99): 1407.0, (109, 100): 1200.0, (109, 101): 1251.0, (109, 102): 901.0, (109, 103): 2048.0, (109, 104): 964.0, (109, 105): 1545.0, (109, 106): 1211.0, (109, 107): 497.0, (109, 108): 2012.0, (109, 109): 0.0, (109, 110): 742.0, (109, 111): 1388.0, (109, 112): 1629.0, (109, 113): 486.0, (109, 114): 549.0, (109, 115): 274.0, (109, 116): 1025.0, (109, 117): 1343.0, (109, 118): 1963.0, (109, 119): 1821.0, (109, 121): 736.0, (109, 122): 1069.0, (109, 123): 1281.0, (109, 124): 1840.0, (109, 125): 1702.0, (109, 126): 1166.0, (109, 127): 1515.0, (109, 128): 1133.0, (109, 129): 1961.0, (109, 130): 628.0, (109, 131): 1945.0, (109, 132): 2535.0, (109, 133): 267.0, (109, 134): 1946.0, (109, 136): 960.0, (109, 137): 751.0, (109, 138): 685.0, (109, 139): 1133.0, (109, 140): 953.0, (109, 141): 1032.0, (109, 142): 419.0, (109, 143): 404.0, (109, 144): 884.0, (109, 145): 150.0, (109, 146): 157.0, (109, 147): 112.0, (109, 148): 674.0, (109, 149): 636.0, (109, 150): 1006.0, (109, 151): 2411.0, (109, 152): 427.0, (109, 153): 1380.0, (109, 154): 1057.0, (109, 155): 970.0, (109, 156): 992.0, (109, 157): 1889.0, (109, 158): 1727.0, (109, 159): 914.0, (109, 160): 959.0, (109, 161): 1974.0, (109, 162): 1380.0, (109, 163): 920.0, (109, 164): 1966.0, (109, 165): 359.0, (109, 166): 1326.0, (109, 167): 641.0, (109, 168): 558.0, (109, 169): 892.0, (109, 170): 138.0, (109, 171): 695.0, (109, 172): 633.0, (109, 173): 799.0, (109, 174): 2052.0, (109, 175): 335.0, (109, 176): 880.0, (109, 177): 1140.0, (109, 178): 509.0, (109, 179): 1125.0, (109, 180): 882.0, (109, 181): 1031.0, (109, 182): 505.0, (109, 183): 2909.0, (109, 184): 1956.0, (109, 185): 1210.0, (109, 186): 377.0, (109, 187): 160.0, (109, 188): 1716.0, (109, 189): 2158.0, (109, 190): 1912.0, (109, 191): 624.0, (109, 192): 899.0, (109, 193): 1955.0, (109, 194): 716.0, (109, 195): 1979.0, (109, 196): 801.0, (109, 197): 1960.0, (109, 198): 1377.0, (109, 199): 1577.0, (109, 200): 1614.0,
                (114, 1): 3493.0, (114, 2): 689.0, (114, 3): 606.0, (114, 4): 1747.0, (114, 5): 536.0, (114, 6): 1667.0, (114, 7): 1082.0, (114, 8): 528.0, (114, 9): 1209.0, (114, 10): 1026.0, (114, 11): 1952.0, (114, 12): 442.0, (114, 13): 656.0, (114, 14): 395.0, (114, 15): 966.0, (114, 16): 1125.0, (114, 17): 1114.0, (114, 18): 779.0, (114, 19): 478.0, (114, 20): 609.0, (114, 21): 1422.0, (114, 22): 582.0, (114, 23): 1620.0, (114, 24): 699.0, (114, 25): 1470.0, (114, 26): 458.0, (114, 27): 1626.0, (114, 28): 338.0, (114, 29): 575.0, (114, 30): 758.0, (114, 31): 389.0, (114, 32): 794.0, (114, 33): 1580.0, (114, 34): 226.0, (114, 35): 1606.0, (114, 36): 714.0, (114, 37): 670.0, (114, 38): 1114.0, (114, 39): 1236.0, (114, 40): 1592.0, (114, 41): 1534.0, (114, 42): 1664.0, (114, 43): 1250.0, (114, 44): 2029.0, (114, 45): 526.0, (114, 46): 416.0, (114, 47): 647.0, (114, 48): 1199.0, (114, 49): 548.0, (114, 50): 546.0, (114, 51): 1507.0, (114, 52): 450.0, (114, 53): 1253.0, (114, 54): 449.0, (114, 55): 921.0, (114, 56): 401.0, (114, 57): 490.0, (114, 58): 301.0, (114, 59): 1413.0, (114, 60): 811.0, (114, 61): 1552.0, (114, 62): 1267.0, (114, 63): 1456.0, (114, 64): 1654.0, (114, 65): 828.0, (114, 66): 647.0, (114, 67): 590.0, (114, 68): 1014.0, (114, 69): 304.0, (114, 70): 678.0, (114, 71): 811.0, (114, 72): 348.0, (114, 73): 836.0, (114, 74): 522.0, (114, 75): 565.0, (114, 76): 812.0, (114, 77): 1639.0, (114, 78): 1360.0, (114, 79): 557.0, (114, 80): 3874.0, (114, 82): 2078.0, (114, 83): 3118.0, (114, 84): 499.0, (114, 85): 1493.0, (114, 86): 894.0, (114, 87): 630.0, (114, 88): 863.0, (114, 89): 211.0, (114, 90): 1685.0, (114, 91): 542.0, (114, 92): 637.0, (114, 93): 728.0, (114, 94): 838.0, (114, 95): 1610.0, (114, 96): 511.0, (114, 97): 1818.0, (114, 98): 1468.0, (114, 99): 1522.0, (114, 100): 1228.0, (114, 101): 920.0, (114, 102): 469.0, (114, 103): 1872.0, (114, 104): 452.0, (114, 105): 1595.0, (114, 106): 697.0, (114, 107): 587.0, (114, 108): 2054.0, (114, 109): 549.0, (114, 110): 690.0, (114, 111): 1119.0, (114, 112): 1189.0, (114, 113): 482.0, (114, 114): 0.0, (114, 115): 433.0, (114, 116): 1290.0, (114, 117): 1394.0, (114, 118): 1636.0, (114, 119): 1444.0, (114, 121): 847.0, (114, 122): 548.0, (114, 123): 873.0, (114, 124): 1454.0, (114, 125): 1408.0, (114, 126): 675.0, (114, 127): 1326.0, (114, 128): 986.0, (114, 129): 1632.0, (114, 130): 692.0, (114, 131): 1619.0, (114, 132): 3026.0, (114, 133): 562.0, (114, 134): 1620.0, (114, 136): 957.0, (114, 137): 860.0, (114, 138): 299.0, (114, 139): 1297.0, (114, 140): 804.0, (114, 141): 531.0, (114, 142): 288.0, (114, 143): 718.0, (114, 144): 400.0, (114, 145): 401.0, (114, 146): 436.0, (114, 147): 437.0, (114, 148): 755.0, (114, 149): 855.0, (114, 150): 616.0, (114, 151): 2897.0, (114, 152): 131.0, (114, 153): 895.0, (114, 154): 829.0, (114, 155): 555.0, (114, 156): 499.0, (114, 157): 1644.0, (114, 158): 1309.0, (114, 159): 1056.0, (114, 160): 878.0, (114, 161): 1645.0, (114, 162): 1150.0, (114, 163): 1071.0, (114, 164): 1635.0, (114, 165): 532.0, (114, 166): 1659.0, (114, 167): 194.0, (114, 168): 588.0, (114, 169): 722.0, (114, 170): 682.0, (114, 171): 633.0, (114, 172): 348.0, (114, 173): 298.0, (114, 174): 1774.0, (114, 175): 632.0, (114, 176): 416.0, (114, 177): 1092.0, (114, 178): 189.0, (114, 179): 1029.0, (114, 180): 1265.0, (114, 181): 846.0, (114, 182): 539.0, (114, 183): 3396.0, (114, 184): 1632.0, (114, 185): 1003.0, (114, 186): 218.0, (114, 187): 707.0, (114, 188): 1308.0, (114, 189): 1913.0, (114, 190): 1633.0, (114, 191): 620.0, (114, 192): 436.0, (114, 193): 1648.0, (114, 194): 585.0, (114, 195): 1665.0, (114, 196): 669.0, (114, 197): 1636.0, (114, 198): 1137.0, (114, 199): 1628.0, (114, 200): 1567.0, (122, 1): 3818.0, (122, 2): 429.0, (122, 3): 1107.0, (122, 4): 1610.0, (122, 5): 850.0, (122, 6): 1267.0, (122, 7): 1603.0, (122, 8): 1008.0, (122, 9): 1381.0, (122, 10): 1460.0, (122, 11): 1638.0, (122, 12): 434.0, (122, 13): 1141.0, (122, 14): 892.0, (122, 15): 1445.0, (122, 16): 972.0, (122, 17): 886.0, (122, 18): 1006.0, (122, 19): 155.0, (122, 20): 1151.0, (122, 21): 965.0, (122, 22): 1127.0, (122, 23): 1658.0, (122, 24): 1247.0, (122, 25): 1438.0, (122, 26): 322.0, (122, 27): 1215.0, (122, 28): 855.0, (122, 29): 1023.0, (122, 30): 1228.0, (122, 31): 838.0, (122, 32): 1232.0, (122, 33): 1187.0, (122, 34): 747.0, (122, 35): 1229.0, (122, 36): 1202.0, (122, 37): 1198.0, (122, 38): 610.0, (122, 39): 866.0, (122, 40): 1212.0, (122, 41): 1278.0, (122, 42): 1523.0, (122, 43): 1622.0, (122, 44): 1668.0, (122, 45): 876.0, (122, 46): 950.0, (122, 47): 165.0, (122, 48): 1674.0, (122, 49): 851.0, (122, 50): 403.0, (122, 51): 1950.0, (122, 52): 894.0, (122, 53): 985.0, (122, 54): 714.0, (122, 55): 1382.0, (122, 56): 535.0, (122, 57): 451.0, (122, 58): 347.0, (122, 59): 947.0, (122, 60): 619.0, (122, 61): 1143.0, (122, 62): 1785.0, (122, 63): 1892.0, (122, 64): 1270.0, (122, 65): 283.0, (122, 66): 887.0, (122, 67): 86.0, (122, 68): 1471.0, (122, 69): 731.0, (122, 70): 931.0, (122, 71): 466.0, (122, 72): 650.0, (122, 73): 1323.0, (122, 74): 1059.0, (122, 75): 1099.0, (122, 76): 551.0, (122, 77): 1226.0, (122, 78): 1766.0, (122, 79): 775.0, (122, 80): 4178.0, (122, 82): 1960.0, (122, 83): 3492.0, (122, 84): 1018.0, (122, 85): 1728.0, (122, 86): 935.0, (122, 87): 1127.0, (122, 88): 1336.0, (122, 89): 518.0, (122, 90): 1427.0, (122, 91): 967.0, (122, 92): 745.0, (122, 93): 922.0, (122, 94): 295.0, (122, 95): 1568.0, (122, 96): 289.0, (122, 97): 1475.0, (122, 98): 1008.0, (122, 99): 1583.0, (122, 100): 1644.0, (122, 101): 608.0, (122, 102): 270.0, (122, 103): 1600.0, (122, 104): 107.0, (122, 105): 1583.0, (122, 106): 150.0, (122, 107): 1107.0, (122, 108): 1990.0, (122, 109): 1069.0, (122, 110): 814.0, (122, 111): 839.0, (122, 112): 688.0, (122, 113): 780.0, (122, 114): 548.0, (122, 115): 981.0, (122, 116): 1804.0, (122, 117): 1810.0, (122, 118): 1225.0, (122, 119): 993.0, (122, 121): 1047.0, (122, 122): 0.0, (122, 123): 456.0, (122, 124): 993.0, (122, 125): 1053.0, (122, 126): 162.0, (122, 127): 1102.0, (122, 128): 888.0, (122, 129): 1219.0, (122, 130): 1188.0, (122, 131): 1209.0, (122, 132): 3401.0, (122, 133): 991.0, (122, 134): 1211.0, (122, 136): 1019.0, (122, 137): 1350.0, (122, 138): 424.0, (122, 139): 1775.0, (122, 140): 770.0, (122, 141): 81.0, (122, 142): 683.0, (122, 143): 1099.0, (122, 144): 204.0, (122, 145): 930.0, (122, 146): 978.0, (122, 147): 959.0, (122, 148): 1248.0, (122, 149): 1378.0, (122, 150): 352.0, (122, 151): 3267.0, (122, 152): 648.0, (122, 153): 365.0, (122, 154): 690.0, (122, 155): 295.0, (122, 156): 118.0, (122, 157): 1320.0, (122, 158): 824.0, (122, 159): 1221.0, (122, 160): 888.0, (122, 161): 1232.0, (122, 162): 909.0, (122, 163): 1242.0, (122, 164): 1220.0, (122, 165): 912.0, (122, 166): 1908.0, (122, 167): 432.0, (122, 168): 839.0, (122, 169): 707.0, (122, 170): 1191.0, (122, 171): 780.0, (122, 172): 524.0, (122, 173): 271.0, (122, 174): 1405.0, (122, 175): 1032.0, (122, 176): 232.0, (122, 177): 1477.0, (122, 178): 700.0, (122, 179): 972.0, (122, 180): 1596.0, (122, 181): 752.0, (122, 182): 1051.0, (122, 183): 3759.0, (122, 184): 1224.0, (122, 185): 829.0, (122, 186): 694.0, (122, 187): 1217.0, (122, 188): 834.0, (122, 189): 1568.0, (122, 190): 1272.0, (122, 191): 825.0, (122, 192): 222.0, (122, 193): 1257.0, (122, 194): 695.0, (122, 195): 1265.0, (122, 196): 724.0, (122, 197): 1229.0, (122, 198): 888.0, (122, 199): 1612.0, (122, 200): 1460.0, (130, 1): 3559.0, (130, 2): 1380.0, (130, 3): 86.0, (130, 4): 2355.0, (130, 5): 1010.0, (130, 6): 2358.0, (130, 7): 427.0, (130, 8): 749.0, (130, 9): 1643.0, (130, 10): 1127.0, (130, 11): 2626.0, (130, 12): 1121.0, (130, 13): 58.0, (130, 14): 680.0, (130, 15): 277.0, (130, 16): 1771.0, (130, 17): 1784.0, (130, 18): 1251.0, (130, 19): 1155.0, (130, 20): 253.0, (130, 21): 2111.0, (130, 22): 470.0, (130, 23): 2124.0, (130, 24): 375.0, (130, 25): 2034.0, (130, 26): 1150.0, (130, 27): 2317.0, (130, 28): 354.0, (130, 29): 202.0, (130, 30): 873.0, (130, 31): 779.0, (130, 32): 206.0, (130, 33): 2270.0, (130, 34): 637.0, (130, 35): 2296.0, (130, 36): 36.0, (130, 37): 151.0, (130, 38): 1788.0, (130, 39): 1926.0, (130, 40): 2281.0, (130, 41): 2198.0, (130, 42): 2277.0, (130, 43): 1420.0, (130, 44): 2714.0, (130, 45): 965.0, (130, 46): 557.0, (130, 47): 1316.0, (130, 48): 508.0, (130, 49): 1029.0, (130, 50): 1233.0, (130, 51): 828.0, (130, 52): 802.0, (130, 53): 1928.0, (130, 54): 1007.0, (130, 55): 991.0, (130, 56): 1051.0, (130, 57): 1168.0, (130, 58): 992.0, (130, 59): 2100.0, (130, 60): 1488.0, (130, 61): 2243.0, (130, 62): 601.0, (130, 63): 783.0, (130, 64): 2343.0, (130, 65): 1469.0, (130, 66): 1148.0, (130, 67): 1249.0, (130, 68): 340.0, (130, 69): 795.0, (130, 70): 1156.0, (130, 71): 1502.0, (130, 72): 929.0, (130, 73): 867.0, (130, 74): 248.0, (130, 75): 210.0, (130, 76): 1499.0, (130, 77): 2331.0, (130, 78): 719.0, (130, 79): 1106.0, (130, 80): 3960.0, (130, 82): 2663.0, (130, 83): 3112.0, (130, 84): 624.0, (130, 85): 1819.0, (130, 86): 1475.0, (130, 87): 62.0, (130, 88): 184.0, (130, 89): 869.0, (130, 90): 2346.0, (130, 91): 867.0, (130, 92): 1232.0, (130, 93): 1241.0, (130, 94): 1481.0, (130, 95): 2170.0, (130, 96): 1203.0, (130, 97): 2501.0, (130, 98): 2156.0, (130, 99): 2016.0, (130, 100): 586.0, (130, 101): 1610.0, (130, 102): 1160.0, (130, 103): 2534.0, (130, 104): 1111.0, (130, 105): 2137.0, (130, 106): 1335.0, (130, 107): 135.0, (130, 108): 2606.0, (130, 109): 628.0, (130, 110): 1263.0, (130, 111): 1800.0, (130, 112): 1864.0, (130, 113): 998.0, (130, 114): 692.0, (130, 115): 383.0, (130, 116): 618.0, (130, 117): 741.0, (130, 118): 2328.0, (130, 119): 2133.0, (130, 121): 1326.0, (130, 122): 1188.0, (130, 123): 1562.0, (130, 124): 2142.0, (130, 125): 2096.0, (130, 126): 1336.0, (130, 127): 1987.0, (130, 128): 1623.0, (130, 129): 2324.0, (130, 130): 0.0, (130, 131): 2310.0, (130, 132): 3025.0, (130, 133): 870.0, (130, 134): 2312.0, (130, 136): 1517.0, (130, 137): 168.0, (130, 138): 982.0, (130, 139): 606.0, (130, 140): 1434.0, (130, 141): 1194.0, (130, 142): 828.0, (130, 143): 1026.0, (130, 144): 1081.0, (130, 145): 579.0, (130, 146): 505.0, (130, 147): 606.0, (130, 148): 63.0, (130, 149): 224.0, (130, 150): 1307.0, (130, 151): 2910.0, (130, 152): 677.0, (130, 153): 1552.0, (130, 154): 1492.0, (130, 155): 1246.0, (130, 156): 1170.0, (130, 157): 2325.0, (130, 158): 1990.0, (130, 159): 1520.0, (130, 160): 1478.0, (130, 161): 2337.0, (130, 162): 1821.0, (130, 163): 1528.0, (130, 164): 2327.0, (130, 165): 931.0, (130, 166): 1947.0, (130, 167): 883.0, (130, 168): 1099.0, (130, 169): 1358.0, (130, 170): 716.0, (130, 171): 1207.0, (130, 172): 1000.0, (130, 173): 979.0, (130, 174): 2462.0, (130, 175): 949.0, (130, 176): 1103.0, (130, 177): 513.0, (130, 178): 506.0, (130, 179): 1643.0, (130, 180): 1499.0, (130, 181): 1494.0, (130, 182): 159.0, (130, 183): 3400.0, (130, 184): 2323.0, (130, 185): 1664.0, (130, 186): 723.0, (130, 187): 719.0, (130, 188): 1993.0, (130, 189): 2595.0, (130, 190): 2320.0, (130, 191): 1157.0, (130, 192): 1122.0, (130, 193): 2338.0, (130, 194): 1192.0, (130, 195): 2356.0, (130, 196): 1283.0, (130, 197): 2328.0, (130, 198): 1811.0, (130, 199): 2170.0, (130, 200): 2168.0,
                (133, 1): 2932.0, (133, 2): 871.0, (133, 3): 801.0, (133, 4): 1536.0, (133, 5): 219.0, (133, 6): 1756.0, (133, 7): 1087.0, (133, 8): 126.0, (133, 9): 773.0, (133, 10): 475.0, (133, 11): 1912.0, (133, 12): 622.0, (133, 13): 872.0, (133, 14): 211.0, (133, 15): 1117.0, (133, 16): 1035.0, (133, 17): 1103.0, (133, 18): 398.0, (133, 19): 857.0, (133, 20): 639.0, (133, 21): 1611.0, (133, 22): 438.0, (133, 23): 1257.0, (133, 24): 603.0, (133, 25): 1190.0, (133, 26): 727.0, (133, 27): 1734.0, (133, 28): 631.0, (133, 29): 886.0, (133, 30): 247.0, (133, 31): 173.0, (133, 32): 1057.0, (133, 33): 1668.0, (133, 34): 360.0, (133, 35): 1674.0, (133, 36): 905.0, (133, 37): 752.0, (133, 38): 1405.0, (133, 39): 1345.0, (133, 40): 1663.0, (133, 41): 1473.0, (133, 42): 1467.0, (133, 43): 688.0, (133, 44): 2040.0, (133, 45): 163.0, (133, 46): 315.0, (133, 47): 1012.0, (133, 48): 1318.0, (133, 49): 234.0, (133, 50): 734.0, (133, 51): 1656.0, (133, 52): 113.0, (133, 53): 1252.0, (133, 54): 325.0, (133, 55): 393.0, (133, 56): 499.0, (133, 57): 643.0, (133, 58): 644.0, (133, 59): 1615.0, (133, 60): 872.0, (133, 61): 1663.0, (133, 62): 1260.0, (133, 63): 1621.0, (133, 64): 1724.0, (133, 65): 1225.0, (133, 66): 331.0, (133, 67): 992.0, (133, 68): 1199.0, (133, 69): 270.0, (133, 70): 319.0, (133, 71): 1007.0, (133, 72): 347.0, (133, 73): 353.0, (133, 74): 622.0, (133, 75): 661.0, (133, 76): 935.0, (133, 77): 1748.0, (133, 78): 1584.0, (133, 79): 356.0, (133, 80): 3315.0, (133, 82): 1822.0, (133, 83): 2556.0, (133, 84): 251.0, (133, 85): 983.0, (133, 86): 679.0, (133, 87): 826.0, (133, 88): 1048.0, (133, 89): 476.0, (133, 90): 1608.0, (133, 91): 24.0, (133, 92): 499.0, (133, 93): 415.0, (133, 94): 1231.0, (133, 95): 1321.0, (133, 96): 791.0, (133, 97): 1825.0, (133, 98): 1656.0, (133, 99): 1148.0, (133, 100): 1452.0, (133, 101): 1049.0, (133, 102): 773.0, (133, 103): 1791.0, (133, 104): 884.0, (133, 105): 1281.0, (133, 106): 1117.0, (133, 107): 735.0, (133, 108): 1748.0, (133, 109): 267.0, (133, 110): 492.0, (133, 111): 1160.0, (133, 112): 1466.0, (133, 113): 265.0, (133, 114): 562.0, (133, 115): 490.0, (133, 116): 1291.0, (133, 117): 1599.0, (133, 118): 1743.0, (133, 119): 1622.0, (133, 121): 469.0, (133, 122): 991.0, (133, 123): 1113.0, (133, 124): 1644.0, (133, 125): 1475.0, (133, 126): 1055.0, (133, 127): 1265.0, (133, 128): 883.0, (133, 129): 1741.0, (133, 130): 870.0, (133, 131): 1725.0, (133, 132): 2465.0, (133, 133): 0.0, (133, 134): 1726.0, (133, 136): 695.0, (133, 137): 1006.0, (133, 138): 570.0, (133, 139): 1398.0, (133, 140): 712.0, (133, 141): 936.0, (133, 142): 311.0, (133, 143): 170.0, (133, 144): 789.0, (133, 145): 297.0, (133, 146): 365.0, (133, 147): 265.0, (133, 148): 922.0, (133, 149): 900.0, (133, 150): 845.0, (133, 151): 2335.0, (133, 152): 434.0, (133, 153): 1253.0, (133, 154): 830.0, (133, 155): 825.0, (133, 156): 893.0, (133, 157): 1646.0, (133, 158): 1549.0, (133, 159): 653.0, (133, 160): 703.0, (133, 161): 1754.0, (133, 162): 1143.0, (133, 163): 660.0, (133, 164): 1747.0, (133, 165): 109.0, (133, 166): 1133.0, (133, 167): 568.0, (133, 168): 308.0, (133, 169): 659.0, (133, 170): 287.0, (133, 171): 452.0, (133, 172): 485.0, (133, 173): 726.0, (133, 174): 1816.0, (133, 175): 82.0, (133, 176): 772.0, (133, 177): 1378.0, (133, 178): 610.0, (133, 179): 868.0, (133, 180): 713.0, (133, 181): 794.0, (133, 182): 730.0, (133, 183): 2834.0, (133, 184): 1735.0, (133, 185): 972.0, (133, 186): 345.0, (133, 187): 311.0, (133, 188): 1533.0, (133, 189): 1913.0, (133, 190): 1678.0, (133, 191): 374.0, (133, 192): 788.0, (133, 193): 1728.0, (133, 194): 490.0, (133, 195): 1754.0, (133, 196): 566.0, (133, 197): 1738.0, (133, 198): 1142.0, (133, 199): 1313.0, (133, 200): 1348.0, (143, 1): 2777.0, (143, 2): 917.0, (143, 3): 961.0, (143, 4): 1433.0, (143, 5): 255.0, (143, 6): 1736.0, (143, 7): 1209.0, (143, 8): 278.0, (143, 9): 620.0, (143, 10): 372.0, (143, 11): 1851.0, (143, 12): 697.0, (143, 13): 1032.0, (143, 14): 381.0, (143, 15): 1263.0, (143, 16): 986.0, (143, 17): 1075.0, (143, 18): 303.0, (143, 19): 956.0, (143, 20): 787.0, (143, 21): 1625.0, (143, 22): 575.0, (143, 23): 1115.0, (143, 24): 731.0, (143, 25): 1074.0, (143, 26): 810.0, (143, 27): 1720.0, (143, 28): 801.0, (143, 29): 1053.0, (143, 30): 254.0, (143, 31): 333.0, (143, 32): 1218.0, (143, 33): 1651.0, (143, 34): 526.0, (143, 35): 1650.0, (143, 36): 1062.0, (143, 37): 901.0, (143, 38): 1455.0, (143, 39): 1343.0, (143, 40): 1640.0, (143, 41): 1415.0, (143, 42): 1368.0, (143, 43): 536.0, (143, 44): 1992.0, (143, 45): 223.0, (143, 46): 479.0, (143, 47): 1099.0, (143, 48): 1456.0, (143, 49): 260.0, (143, 50): 796.0, (143, 51): 1794.0, (143, 52): 277.0, (143, 53): 1220.0, (143, 54): 391.0, (143, 55): 337.0, (143, 56): 577.0, (143, 57): 707.0, (143, 58): 754.0, (143, 59): 1632.0, (143, 60): 882.0, (143, 61): 1653.0, (143, 62): 1375.0, (143, 63): 1762.0, (143, 64): 1700.0, (143, 65): 1312.0, (143, 66): 297.0, (143, 67): 1089.0, (143, 68): 1349.0, (143, 69): 415.0, (143, 70): 265.0, (143, 71): 1047.0, (143, 72): 450.0, (143, 73): 355.0, (143, 74): 780.0, (143, 75): 817.0, (143, 76): 958.0, (143, 77): 1735.0, (143, 78): 1734.0, (143, 79): 372.0, (143, 80): 3156.0, (143, 82): 1701.0, (143, 83): 2413.0, (143, 84): 402.0, (143, 85): 813.0, (143, 86): 614.0, (143, 87): 985.0, (143, 88): 1201.0, (143, 89): 602.0, (143, 90): 1542.0, (143, 91): 184.0, (143, 92): 496.0, (143, 93): 355.0, (143, 94): 1317.0, (143, 95): 1199.0, (143, 96): 870.0, (143, 97): 1780.0, (143, 98): 1668.0, (143, 99): 1005.0, (143, 100): 1604.0, (143, 101): 1067.0, (143, 102): 860.0, (143, 103): 1720.0, (143, 104): 994.0, (143, 105): 1152.0, (143, 106): 1214.0, (143, 107): 893.0, (143, 108): 1615.0, (143, 109): 404.0, (143, 110): 465.0, (143, 111): 1145.0, (143, 112): 1511.0, (143, 113): 324.0, (143, 114): 718.0, (143, 115): 653.0, (143, 116): 1406.0, (143, 117): 1747.0, (143, 118): 1729.0, (143, 119): 1632.0, (143, 121): 362.0, (143, 122): 1099.0, (143, 123): 1159.0, (143, 124): 1658.0, (143, 125): 1456.0, (143, 126): 1144.0, (143, 127): 1214.0, (143, 128): 839.0, (143, 129): 1728.0, (143, 130): 1026.0, (143, 131): 1711.0, (143, 132): 2321.0, (143, 133): 170.0, (143, 134): 1712.0, (143, 136): 610.0, (143, 137): 1155.0, (143, 138): 675.0, (143, 139): 1531.0, (143, 140): 688.0, (143, 141): 1037.0, (143, 142): 446.0, (143, 143): 0.0, (143, 144): 896.0, (143, 145): 463.0, (143, 146): 526.0, (143, 147): 428.0, (143, 148): 1076.0, (143, 149): 1037.0, (143, 150): 906.0, (143, 151): 2190.0, (143, 152): 593.0, (143, 153): 1328.0, (143, 154): 825.0, (143, 155): 899.0, (143, 156): 994.0, (143, 157): 1603.0, (143, 158): 1580.0, (143, 159): 517.0, (143, 160): 651.0, (143, 161): 1740.0, (143, 162): 1114.0, (143, 163): 520.0, (143, 164): 1734.0, (143, 165): 193.0, (143, 166): 963.0, (143, 167): 695.0, (143, 168): 307.0, (143, 169): 653.0, (143, 170): 371.0, (143, 171): 443.0, (143, 172): 577.0, (143, 173): 845.0, (143, 174): 1781.0, (143, 175): 89.0, (143, 176): 871.0, (143, 177): 1537.0, (143, 178): 779.0, (143, 179): 805.0, (143, 180): 548.0, (143, 181): 776.0, (143, 182): 892.0, (143, 183): 2688.0, (143, 184): 1720.0, (143, 185): 945.0, (143, 186): 501.0, (143, 187): 387.0, (143, 188): 1560.0, (143, 189): 1864.0, (143, 190): 1647.0, (143, 191): 361.0, (143, 192): 886.0, (143, 193): 1706.0, (143, 194): 506.0, (143, 195): 1735.0, (143, 196): 561.0, (143, 197): 1723.0, (143, 198): 1117.0, (143, 199): 1183.0, (143, 200): 1246.0, (147, 1): 3123.0, (147, 2): 959.0, (147, 3): 536.0, (147, 4): 1774.0, (147, 5): 422.0, (147, 6): 1905.0, (147, 7): 856.0, (147, 8): 151.0, (147, 9): 1038.0, (147, 10): 631.0, (147, 11): 2107.0, (147, 12): 687.0, (147, 13): 607.0, (147, 14): 94.0, (147, 15): 858.0, (147, 16): 1231.0, (147, 17): 1276.0, (147, 18): 649.0, (147, 19): 854.0, (147, 20): 383.0, (147, 21): 1719.0, (147, 22): 213.0, (147, 23): 1519.0, (147, 24): 379.0, (147, 25): 1439.0, (147, 26): 765.0, (147, 27): 1876.0, (147, 28): 385.0, (147, 29): 626.0, (147, 30): 350.0, (147, 31): 202.0, (147, 32): 792.0, (147, 33): 1817.0, (147, 34): 215.0, (147, 35): 1830.0, (147, 36): 641.0, (147, 37): 493.0, (147, 38): 1463.0, (147, 39): 1480.0, (147, 40): 1818.0, (147, 41): 1669.0, (147, 42): 1701.0, (147, 43): 894.0, (147, 44): 2220.0, (147, 45): 370.0, (147, 46): 51.0, (147, 47): 1023.0, (147, 48): 1067.0, (147, 49): 440.0, (147, 50): 809.0, (147, 51): 1404.0, (147, 52): 204.0, (147, 53): 1425.0, (147, 54): 460.0, (147, 55): 509.0, (147, 56): 577.0, (147, 57): 723.0, (147, 58): 638.0, (147, 59): 1717.0, (147, 60): 1008.0, (147, 61): 1803.0, (147, 62): 1036.0, (147, 63): 1366.0, (147, 64): 1880.0, (147, 65): 1225.0, (147, 66): 557.0, (147, 67): 983.0, (147, 68): 938.0, (147, 69): 274.0, (147, 70): 558.0, (147, 71): 1096.0, (147, 72): 418.0, (147, 73): 409.0, (147, 74): 358.0, (147, 75): 398.0, (147, 76): 1050.0, (147, 77): 1890.0, (147, 78): 1322.0, (147, 79): 539.0, (147, 80): 3514.0, (147, 82): 2071.0, (147, 83): 2722.0, (147, 84): 62.0, (147, 85): 1237.0, (147, 86): 900.0, (147, 87): 561.0, (147, 88): 785.0, (147, 89): 461.0, (147, 90): 1811.0, (147, 91): 262.0, (147, 92): 680.0, (147, 93): 649.0, (147, 94): 1234.0, (147, 95): 1573.0, (147, 96): 830.0, (147, 97): 2005.0, (147, 98): 1764.0, (147, 99): 1410.0, (147, 100): 1190.0, (147, 101): 1167.0, (147, 102): 800.0, (147, 103): 1997.0, (147, 104): 855.0, (147, 105): 1536.0, (147, 106): 1102.0, (147, 107): 471.0, (147, 108): 2005.0, (147, 109): 112.0, (147, 110): 694.0, (147, 111): 1316.0, (147, 112): 1532.0, (147, 113): 428.0, (147, 114): 437.0, (147, 115): 226.0, (147, 116): 1065.0, (147, 117): 1339.0, (147, 118): 1885.0, (147, 119): 1734.0, (147, 121): 723.0, (147, 122): 959.0, (147, 123): 1187.0, (147, 124): 1752.0, (147, 125): 1628.0, (147, 126): 1060.0, (147, 127): 1458.0, (147, 128): 1079.0, (147, 129): 1883.0, (147, 130): 606.0, (147, 131): 1867.0, (147, 132): 2631.0, (147, 133): 265.0, (147, 134): 1869.0, (147, 136): 930.0, (147, 137): 746.0, (147, 138): 584.0, (147, 139): 1151.0, (147, 140): 895.0, (147, 141): 924.0, (147, 142): 326.0, (147, 143): 428.0, (147, 144): 777.0, (147, 145): 43.0, (147, 146): 101.0, (147, 147): 0.0, (147, 148): 659.0, (147, 149): 654.0, (147, 150): 910.0, (147, 151): 2506.0, (147, 152): 315.0, (147, 153): 1275.0, (147, 154): 987.0, (147, 155): 871.0, (147, 156): 885.0, (147, 157): 1825.0, (147, 158): 1633.0, (147, 159): 915.0, (147, 160): 913.0, (147, 161): 1896.0, (147, 162): 1315.0, (147, 163): 923.0, (147, 164): 1888.0, (147, 165): 329.0, (147, 166): 1380.0, (147, 167): 533.0, (147, 168): 516.0, (147, 169): 827.0, (147, 170): 247.0, (147, 171): 643.0, (147, 172): 540.0, (147, 173): 689.0, (147, 174): 1983.0, (147, 175): 345.0, (147, 176): 774.0, (147, 177): 1113.0, (147, 178): 407.0, (147, 179): 1080.0, (147, 180): 944.0, (147, 181): 968.0, (147, 182): 466.0, (147, 183): 3005.0, (147, 184): 1879.0, (147, 185): 1147.0, (147, 186): 270.0, (147, 187): 270.0, (147, 188): 1624.0, (147, 189): 2096.0, (147, 190): 1842.0, (147, 191): 579.0, (147, 192): 794.0, (147, 193): 1880.0, (147, 194): 652.0, (147, 195): 1904.0, (147, 196): 741.0, (147, 197): 1883.0, (147, 198): 1310.0, (147, 199): 1569.0, (147, 200): 1586.0,
                (151, 1): 669.0, (151, 2): 2976.0, (151, 3): 2892.0, (151, 4): 2555.0, (151, 5): 2423.0, (151, 6): 3375.0, (151, 7): 2768.0, (151, 8): 2379.0, (151, 9): 1978.0, (151, 10): 1877.0, (151, 11): 3174.0, (151, 12): 2838.0, (151, 13): 2949.0, (151, 14): 2514.0, (151, 15): 3000.0, (151, 16): 2692.0, (151, 17): 2846.0, (151, 18): 2270.0, (151, 19): 3116.0, (151, 20): 2677.0, (151, 21): 3463.0, (151, 22): 2497.0, (151, 23): 2056.0, (151, 24): 2538.0, (151, 25): 2306.0, (151, 26): 2953.0, (151, 27): 3398.0, (151, 28): 2872.0, (151, 29): 3044.0, (151, 30): 2156.0, (151, 31): 2508.0, (151, 32): 3109.0, (151, 33): 3319.0, (151, 34): 2683.0, (151, 35): 3281.0, (151, 36): 2942.0, (151, 37): 2760.0, (151, 38): 3462.0, (151, 39): 3160.0, (151, 40): 3282.0, (151, 41): 2905.0, (151, 42): 2572.0, (151, 43): 1655.0, (151, 44): 3377.0, (151, 45): 2407.0, (151, 46): 2549.0, (151, 47): 3230.0, (151, 48): 3059.0, (151, 49): 2420.0, (151, 50): 2904.0, (151, 51): 3308.0, (151, 52): 2448.0, (151, 53): 2926.0, (151, 54): 2557.0, (151, 55): 1997.0, (151, 56): 2732.0, (151, 57): 2831.0, (151, 58): 2932.0, (151, 59): 3487.0, (151, 60): 2850.0, (151, 61): 3360.0, (151, 62): 2806.0, (151, 63): 3317.0, (151, 64): 3317.0, (151, 65): 3435.0, (151, 66): 2381.0, (151, 67): 3241.0, (151, 68): 3095.0, (151, 69): 2602.0, (151, 70): 2337.0, (151, 71): 3072.0, (151, 72): 2634.0, (151, 73): 2106.0, (151, 74): 2736.0, (151, 75): 2745.0, (151, 76): 2950.0, (151, 77): 3410.0, (151, 78): 3398.0, (151, 79): 2492.0, (151, 80): 1076.0, (151, 82): 2493.0, (151, 83): 226.0, (151, 84): 2449.0, (151, 85): 1615.0, (151, 86): 2444.0, (151, 87): 2905.0, (151, 88): 3009.0, (151, 89): 2792.0, (151, 90): 2929.0, (151, 91): 2357.0, (151, 92): 2545.0, (151, 93): 2357.0, (151, 94): 3437.0, (151, 95): 2289.0, (151, 96): 3004.0, (151, 97): 3238.0, (151, 98): 3493.0, (151, 99): 2033.0, (151, 100): 3312.0, (151, 101): 3018.0, (151, 102): 3006.0, (151, 103): 3006.0, (151, 104): 3167.0, (151, 105): 2209.0, (151, 106): 3361.0, (151, 107): 2819.0, (151, 108): 2295.0, (151, 109): 2411.0, (151, 110): 2479.0, (151, 111): 2953.0, (151, 112): 3492.0, (151, 113): 2494.0, (151, 114): 2897.0, (151, 115): 2669.0, (151, 116): 2834.0, (151, 117): 3377.0, (151, 118): 3402.0, (151, 119): 3453.0, (151, 121): 2242.0, (151, 122): 3267.0, (151, 123): 3188.0, (151, 124): 3489.0, (151, 125): 3162.0, (151, 126): 3278.0, (151, 127): 2821.0, (151, 128): 2633.0, (151, 129): 3406.0, (151, 130): 2910.0, (151, 131): 3390.0, (151, 132): 136.0, (151, 133): 2335.0, (151, 134): 3390.0, (151, 136): 2367.0, (151, 137): 2939.0, (151, 138): 2852.0, (151, 139): 3068.0, (151, 140): 2617.0, (151, 141): 3197.0, (151, 142): 2636.0, (151, 143): 2190.0, (151, 144): 3066.0, (151, 145): 2549.0, (151, 146): 2566.0, (151, 147): 2506.0, (151, 148): 2924.0, (151, 149): 2766.0, (151, 150): 3001.0, (151, 151): 0.0, (151, 152): 2767.0, (151, 153): 3421.0, (151, 154): 2763.0, (151, 155): 3020.0, (151, 156): 3154.0, (151, 157): 3127.0, (151, 158): 3501.0, (151, 159): 2114.0, (151, 160): 2504.0, (151, 161): 3413.0, (151, 162): 2869.0, (151, 163): 2093.0, (151, 164): 3415.0, (151, 165): 2383.0, (151, 166): 1441.0, (151, 167): 2885.0, (151, 168): 2428.0, (151, 169): 2641.0, (151, 170): 2277.0, (151, 171): 2501.0, (151, 172): 2747.0, (151, 173): 3029.0, (151, 174): 3312.0, (151, 175): 2271.0, (151, 176): 3035.0, (151, 177): 3354.0, (151, 178): 2913.0, (151, 179): 2537.0, (151, 180): 1672.0, (151, 181): 2686.0, (151, 182): 2855.0, (151, 183): 499.0, (151, 184): 3390.0, (151, 185): 2765.0, (151, 186): 2680.0, (151, 187): 2261.0, (151, 188): 3466.0, (151, 189): 3274.0, (151, 190): 3236.0, (151, 191): 2446.0, (151, 192): 3046.0, (151, 193): 3339.0, (151, 194): 2587.0, (151, 195): 3375.0, (151, 196): 2585.0, (151, 197): 3390.0, (151, 198): 2888.0, (151, 199): 2208.0, (151, 200): 2486.0, (154, 1): 3256.0, (154, 2): 266.0, (154, 3): 1406.0, (154, 4): 946.0, (154, 5): 611.0, (154, 6): 926.0, (154, 7): 1828.0, (154, 8): 928.0, (154, 9): 789.0, (154, 10): 1176.0, (154, 11): 1135.0, (154, 12): 391.0, (154, 13): 1466.0, (154, 14): 892.0, (154, 15): 1769.0, (154, 16): 303.0, (154, 17): 293.0, (154, 18): 567.0, (154, 19): 558.0, (154, 20): 1337.0, (154, 21): 803.0, (154, 22): 1199.0, (154, 23): 975.0, (154, 24): 1364.0, (154, 25): 748.0, (154, 26): 427.0, (154, 27): 903.0, (154, 28): 1145.0, (154, 29): 1401.0, (154, 30): 1059.0, (154, 31): 786.0, (154, 32): 1618.0, (154, 33): 838.0, (154, 34): 862.0, (154, 35): 847.0, (154, 36): 1519.0, (154, 37): 1430.0, (154, 38): 701.0, (154, 39): 518.0, (154, 40): 835.0, (154, 41): 707.0, (154, 42): 860.0, (154, 43): 1215.0, (154, 44): 1235.0, (154, 45): 667.0, (154, 46): 1013.0, (154, 47): 566.0, (154, 48): 1997.0, (154, 49): 597.0, (154, 50): 319.0, (154, 51): 2319.0, (154, 52): 804.0, (154, 53): 440.0, (154, 54): 528.0, (154, 55): 1162.0, (154, 56): 441.0, (154, 57): 343.0, (154, 58): 558.0, (154, 59): 813.0, (154, 60): 89.0, (154, 61): 833.0, (154, 62): 2013.0, (154, 63): 2272.0, (154, 64): 897.0, (154, 65): 725.0, (154, 66): 528.0, (154, 67): 620.0, (154, 68): 1827.0, (154, 69): 722.0, (154, 70): 563.0, (154, 71): 316.0, (154, 72): 576.0, (154, 73): 1167.0, (154, 74): 1275.0, (154, 75): 1322.0, (154, 76): 190.0, (154, 77): 918.0, (154, 78): 2186.0, (154, 79): 475.0, (154, 80): 3593.0, (154, 82): 1291.0, (154, 83): 2989.0, (154, 84): 1017.0, (154, 85): 1155.0, (154, 86): 319.0, (154, 87): 1432.0, (154, 88): 1673.0, (154, 89): 627.0, (154, 90): 857.0, (154, 91): 812.0, (154, 92): 333.0, (154, 93): 486.0, (154, 94): 722.0, (154, 95): 878.0, (154, 96): 420.0, (154, 97): 1020.0, (154, 98): 845.0, (154, 99): 910.0, (154, 100): 2053.0, (154, 101): 262.0, (154, 102): 458.0, (154, 103): 1045.0, (154, 104): 642.0, (154, 105): 893.0, (154, 106): 703.0, (154, 107): 1369.0, (154, 108): 1305.0, (154, 109): 1057.0, (154, 110): 362.0, (154, 111): 331.0, (154, 112): 738.0, (154, 113): 573.0, (154, 114): 829.0, (154, 115): 1146.0, (154, 116): 2040.0, (154, 117): 2217.0, (154, 118): 912.0, (154, 119): 809.0, (154, 121): 558.0, (154, 122): 690.0, (154, 123): 428.0, (154, 124): 836.0, (154, 125): 646.0, (154, 126): 608.0, (154, 127): 498.0, (154, 128): 198.0, (154, 129): 911.0, (154, 130): 1492.0, (154, 131): 895.0, (154, 132): 2899.0, (154, 133): 830.0, (154, 134): 896.0, (154, 136): 398.0, (154, 137): 1657.0, (154, 138): 532.0, (154, 139): 2091.0, (154, 140): 152.0, (154, 141): 610.0, (154, 142): 678.0, (154, 143): 825.0, (154, 144): 574.0, (154, 145): 985.0, (154, 146): 1066.0, (154, 147): 987.0, (154, 148): 1555.0, (154, 149): 1611.0, (154, 150): 338.0, (154, 151): 2763.0, (154, 152): 815.0, (154, 153): 680.0, (154, 154): 0.0, (154, 155): 400.0, (154, 156): 584.0, (154, 157): 841.0, (154, 158): 778.0, (154, 159): 649.0, (154, 160): 259.0, (154, 161): 923.0, (154, 162): 331.0, (154, 163): 671.0, (154, 164): 917.0, (154, 165): 721.0, (154, 166): 1336.0, (154, 167): 640.0, (154, 168): 528.0, (154, 169): 173.0, (154, 170): 1115.0, (154, 171): 385.0, (154, 172): 492.0, (154, 173): 633.0, (154, 174): 997.0, (154, 175): 813.0, (154, 176): 529.0, (154, 177): 1921.0, (154, 178): 1013.0, (154, 179): 286.0, (154, 180): 1138.0, (154, 181): 78.0, (154, 182): 1333.0, (154, 183): 3237.0, (154, 184): 905.0, (154, 185): 175.0, (154, 186): 775.0, (154, 187): 1140.0, (154, 188): 752.0, (154, 189): 1112.0, (154, 190): 856.0, (154, 191): 467.0, (154, 192): 522.0, (154, 193): 899.0, (154, 194): 342.0, (154, 195): 925.0, (154, 196): 266.0, (154, 197): 908.0, (154, 198): 324.0, (154, 199): 922.0, (154, 200): 782.0, (159, 1): 2618.0, (159, 2): 872.0, (159, 3): 1448.0, (159, 4): 935.0, (159, 5): 534.0, (159, 6): 1409.0, (159, 7): 1726.0, (159, 8): 778.0, (159, 9): 161.0, (159, 10): 711.0, (159, 11): 1421.0, (159, 12): 794.0, (159, 13): 1518.0, (159, 14): 843.0, (159, 15): 1770.0, (159, 16): 639.0, (159, 17): 777.0, (159, 18): 281.0, (159, 19): 1067.0, (159, 20): 1292.0, (159, 21): 1396.0, (159, 22): 1087.0, (159, 23): 604.0, (159, 24): 1247.0, (159, 25): 566.0, (159, 26): 900.0, (159, 27): 1410.0, (159, 28): 1248.0, (159, 29): 1516.0, (159, 30): 754.0, (159, 31): 754.0, (159, 32): 1702.0, (159, 33): 1332.0, (159, 34): 937.0, (159, 35): 1315.0, (159, 36): 1555.0, (159, 37): 1405.0, (159, 38): 1350.0, (159, 39): 1091.0, (159, 40): 1309.0, (159, 41): 1008.0, (159, 42): 880.0, (159, 43): 627.0, (159, 44): 1586.0, (159, 45): 567.0, (159, 46): 963.0, (159, 47): 1148.0, (159, 48): 1969.0, (159, 49): 518.0, (159, 50): 825.0, (159, 51): 2307.0, (159, 52): 721.0, (159, 53): 894.0, (159, 54): 607.0, (159, 55): 767.0, (159, 56): 723.0, (159, 57): 771.0, (159, 58): 937.0, (159, 59): 1415.0, (159, 60): 736.0, (159, 61): 1353.0, (159, 62): 1891.0, (159, 63): 2274.0, (159, 64): 1361.0, (159, 65): 1340.0, (159, 66): 409.0, (159, 67): 1175.0, (159, 68): 1852.0, (159, 69): 780.0, (159, 70): 383.0, (159, 71): 959.0, (159, 72): 709.0, (159, 73): 837.0, (159, 74): 1273.0, (159, 75): 1313.0, (159, 76): 836.0, (159, 77): 1424.0, (159, 78): 2236.0, (159, 79): 503.0, (159, 80): 2967.0, (159, 82): 1186.0, (159, 83): 2340.0, (159, 84): 904.0, (159, 85): 514.0, (159, 86): 331.0, (159, 87): 1473.0, (159, 88): 1700.0, (159, 89): 866.0, (159, 90): 1111.0, (159, 91): 654.0, (159, 92): 479.0, (159, 93): 330.0, (159, 94): 1340.0, (159, 95): 685.0, (159, 96): 936.0, (159, 97): 1386.0, (159, 98): 1433.0, (159, 99): 495.0, (159, 100): 2105.0, (159, 101): 907.0, (159, 102): 951.0, (159, 103): 1274.0, (159, 104): 1136.0, (159, 105): 635.0, (159, 106): 1286.0, (159, 107): 1385.0, (159, 108): 1099.0, (159, 109): 914.0, (159, 110): 410.0, (159, 111): 876.0, (159, 112): 1383.0, (159, 113): 576.0, (159, 114): 1056.0, (159, 115): 1138.0, (159, 116): 1923.0, (159, 117): 2253.0, (159, 118): 1416.0, (159, 119): 1394.0, (159, 121): 209.0, (159, 122): 1221.0, (159, 123): 1074.0, (159, 124): 1427.0, (159, 125): 1145.0, (159, 126): 1195.0, (159, 127): 837.0, (159, 128): 539.0, (159, 129): 1418.0, (159, 130): 1520.0, (159, 131): 1400.0, (159, 132): 2249.0, (159, 133): 653.0, (159, 134): 1401.0, (159, 136): 257.0, (159, 137): 1659.0, (159, 138): 864.0, (159, 139): 2046.0, (159, 140): 504.0, (159, 141): 1143.0, (159, 142): 776.0, (159, 143): 517.0, (159, 144): 1039.0, (159, 145): 941.0, (159, 146): 1016.0, (159, 147): 915.0, (159, 148): 1574.0, (159, 149): 1550.0, (159, 150): 909.0, (159, 151): 2114.0, (159, 152): 962.0, (159, 153): 1314.0, (159, 154): 649.0, (159, 155): 942.0, (159, 156): 1104.0, (159, 157): 1222.0, (159, 158): 1406.0, (159, 159): 0.0, (159, 160): 391.0, (159, 161): 1428.0, (159, 162): 808.0, (159, 163): 21.0, (159, 164): 1426.0, (159, 165): 593.0, (159, 166): 696.0, (159, 167): 941.0, (159, 168): 468.0, (159, 169): 536.0, (159, 170): 887.0, (159, 171): 453.0, (159, 172): 760.0, (159, 173): 1040.0, (159, 174): 1412.0, (159, 175): 579.0, (159, 176): 999.0, (159, 177): 2023.0, (159, 178): 1181.0, (159, 179): 455.0, (159, 180): 521.0, (159, 181): 573.0, (159, 182): 1375.0, (159, 183): 2590.0, (159, 184): 1406.0, (159, 185): 672.0, (159, 186): 874.0, (159, 187): 901.0, (159, 188): 1375.0, (159, 189): 1459.0, (159, 190): 1294.0, (159, 191): 440.0, (159, 192): 1004.0, (159, 193): 1375.0, (159, 194): 533.0, (159, 195): 1408.0, (159, 196): 498.0, (159, 197): 1408.0, (159, 198): 822.0, (159, 199): 666.0, (159, 200): 754.0,
                (169, 1): 3154.0, (169, 2): 336.0, (169, 3): 1273.0, (169, 4): 1024.0, (169, 5): 440.0, (169, 6): 1097.0, (169, 7): 1677.0, (169, 8): 760.0, (169, 9): 690.0, (169, 10): 1007.0, (169, 11): 1281.0, (169, 12): 319.0, (169, 13): 1335.0, (169, 14): 733.0, (169, 15): 1633.0, (169, 16): 413.0, (169, 17): 449.0, (169, 18): 405.0, (169, 19): 558.0, (169, 20): 1188.0, (169, 21): 973.0, (169, 22): 1040.0, (169, 23): 962.0, (169, 24): 1206.0, (169, 25): 769.0, (169, 26): 398.0, (169, 27): 1076.0, (169, 28): 1019.0, (169, 29): 1283.0, (169, 30): 886.0, (169, 31): 625.0, (169, 32): 1496.0, (169, 33): 1009.0, (169, 34): 721.0, (169, 35): 1015.0, (169, 36): 1387.0, (169, 37): 1286.0, (169, 38): 842.0, (169, 39): 692.0, (169, 40): 1004.0, (169, 41): 846.0, (169, 42): 941.0, (169, 43): 1061.0, (169, 44): 1393.0, (169, 45): 496.0, (169, 46): 857.0, (169, 47): 616.0, (169, 48): 1858.0, (169, 49): 425.0, (169, 50): 304.0, (169, 51): 2185.0, (169, 52): 639.0, (169, 53): 599.0, (169, 54): 367.0, (169, 55): 990.0, (169, 56): 321.0, (169, 57): 279.0, (169, 58): 490.0, (169, 59): 982.0, (169, 60): 234.0, (169, 61): 1006.0, (169, 62): 1860.0, (169, 63): 2140.0, (169, 64): 1066.0, (169, 65): 804.0, (169, 66): 355.0, (169, 67): 651.0, (169, 68): 1696.0, (169, 69): 571.0, (169, 70): 392.0, (169, 71): 435.0, (169, 72): 429.0, (169, 73): 994.0, (169, 74): 1132.0, (169, 75): 1178.0, (169, 76): 323.0, (169, 77): 1090.0, (169, 78): 2063.0, (169, 79): 303.0, (169, 80): 3502.0, (169, 82): 1359.0, (169, 83): 2867.0, (169, 84): 854.0, (169, 85): 1050.0, (169, 86): 228.0, (169, 87): 1299.0, (169, 88): 1541.0, (169, 89): 513.0, (169, 90): 991.0, (169, 91): 641.0, (169, 92): 160.0, (169, 93): 320.0, (169, 94): 804.0, (169, 95): 907.0, (169, 96): 418.0, (169, 97): 1178.0, (169, 98): 1016.0, (169, 99): 881.0, (169, 100): 1929.0, (169, 101): 421.0, (169, 102): 443.0, (169, 103): 1179.0, (169, 104): 636.0, (169, 105): 905.0, (169, 106): 755.0, (169, 107): 1230.0, (169, 108): 1348.0, (169, 109): 892.0, (169, 110): 192.0, (169, 111): 502.0, (169, 112): 887.0, (169, 113): 405.0, (169, 114): 722.0, (169, 115): 999.0, (169, 116): 1888.0, (169, 117): 2091.0, (169, 118): 1084.0, (169, 119): 980.0, (169, 121): 408.0, (169, 122): 707.0, (169, 123): 553.0, (169, 124): 1006.0, (169, 125): 816.0, (169, 126): 662.0, (169, 127): 634.0, (169, 128): 266.0, (169, 129): 1083.0, (169, 130): 1358.0, (169, 131): 1067.0, (169, 132): 2777.0, (169, 133): 659.0, (169, 134): 1068.0, (169, 136): 313.0, (169, 137): 1520.0, (169, 138): 442.0, (169, 139): 1950.0, (169, 140): 82.0, (169, 141): 626.0, (169, 142): 532.0, (169, 143): 653.0, (169, 144): 549.0, (169, 145): 830.0, (169, 146): 911.0, (169, 147): 827.0, (169, 148): 1420.0, (169, 149): 1464.0, (169, 150): 375.0, (169, 151): 2641.0, (169, 152): 687.0, (169, 153): 780.0, (169, 154): 173.0, (169, 155): 415.0, (169, 156): 591.0, (169, 157): 998.0, (169, 158): 939.0, (169, 159): 536.0, (169, 160): 183.0, (169, 161): 1096.0, (169, 162): 489.0, (169, 163): 558.0, (169, 164): 1089.0, (169, 165): 550.0, (169, 166): 1232.0, (169, 167): 547.0, (169, 168): 355.0, (169, 169): 0.0, (169, 170): 945.0, (169, 171): 211.0, (169, 172): 374.0, (169, 173): 582.0, (169, 174): 1161.0, (169, 175): 639.0, (169, 176): 504.0, (169, 177): 1806.0, (169, 178): 898.0, (169, 179): 309.0, (169, 180): 994.0, (169, 181): 140.0, (169, 182): 1199.0, (169, 183): 3122.0, (169, 184): 1077.0, (169, 185): 319.0, (169, 186): 635.0, (169, 187): 970.0, (169, 188): 916.0, (169, 189): 1268.0, (169, 190): 1021.0, (169, 191): 293.0, (169, 192): 504.0, (169, 193): 1069.0, (169, 194): 176.0, (169, 195): 1096.0, (169, 196): 93.0, (169, 197): 1080.0, (169, 198): 486.0, (169, 199): 937.0, (169, 200): 846.0, (191, 1): 2994.0, (191, 2): 568.0, (191, 3): 1077.0, (191, 4): 1200.0, (191, 5): 160.0, (191, 6): 1387.0, (191, 7): 1434.0, (191, 8): 485.0, (191, 9): 598.0, (191, 10): 724.0, (191, 11): 1540.0, (191, 12): 392.0, (191, 13): 1144.0, (191, 14): 489.0, (191, 15): 1424.0, (191, 16): 663.0, (191, 17): 731.0, (191, 18): 182.0, (191, 19): 672.0, (191, 20): 958.0, (191, 21): 1264.0, (191, 22): 784.0, (191, 23): 1005.0, (191, 24): 953.0, (191, 25): 880.0, (191, 26): 508.0, (191, 27): 1368.0, (191, 28): 847.0, (191, 29): 1118.0, (191, 30): 593.0, (191, 31): 382.0, (191, 32): 1319.0, (191, 33): 1300.0, (191, 34): 532.0, (191, 35): 1303.0, (191, 36): 1189.0, (191, 37): 1064.0, (191, 38): 1104.0, (191, 39): 985.0, (191, 40): 1293.0, (191, 41): 1101.0, (191, 42): 1124.0, (191, 43): 816.0, (191, 44): 1666.0, (191, 45): 216.0, (191, 46): 617.0, (191, 47): 788.0, (191, 48): 1641.0, (191, 49): 142.0, (191, 50): 465.0, (191, 51): 1975.0, (191, 52): 377.0, (191, 53): 879.0, (191, 54): 171.0, (191, 55): 698.0, (191, 56): 296.0, (191, 57): 386.0, (191, 58): 510.0, (191, 59): 1271.0, (191, 60): 521.0, (191, 61): 1299.0, (191, 62): 1613.0, (191, 63): 1934.0, (191, 64): 1353.0, (191, 65): 997.0, (191, 66): 69.0, (191, 67): 795.0, (191, 68): 1496.0, (191, 69): 368.0, (191, 70): 113.0, (191, 71): 693.0, (191, 72): 272.0, (191, 73): 701.0, (191, 74): 916.0, (191, 75): 961.0, (191, 76): 598.0, (191, 77): 1382.0, (191, 78): 1875.0, (191, 79): 63.0, (191, 80): 3357.0, (191, 82): 1507.0, (191, 83): 2671.0, (191, 84): 593.0, (191, 85): 916.0, (191, 86): 322.0, (191, 87): 1103.0, (191, 88): 1341.0, (191, 89): 426.0, (191, 90): 1239.0, (191, 91): 359.0, (191, 92): 135.0, (191, 93): 110.0, (191, 94): 1000.0, (191, 95): 1018.0, (191, 96): 559.0, (191, 97): 1451.0, (191, 98): 1308.0, (191, 99): 905.0, (191, 100): 1741.0, (191, 101): 706.0, (191, 102): 561.0, (191, 103): 1424.0, (191, 104): 729.0, (191, 105): 993.0, (191, 106): 916.0, (191, 107): 1023.0, (191, 108): 1460.0, (191, 109): 624.0, (191, 110): 118.0, (191, 111): 792.0, (191, 112): 1156.0, (191, 113): 159.0, (191, 114): 620.0, (191, 115): 779.0, (191, 116): 1643.0, (191, 117): 1898.0, (191, 118): 1376.0, (191, 119): 1272.0, (191, 121): 236.0, (191, 122): 825.0, (191, 123): 808.0, (191, 124): 1297.0, (191, 125): 1106.0, (191, 126): 835.0, (191, 127): 893.0, (191, 128): 510.0, (191, 129): 1375.0, (191, 130): 1157.0, (191, 131): 1359.0, (191, 132): 2580.0, (191, 133): 374.0, (191, 134): 1360.0, (191, 136): 361.0, (191, 137): 1311.0, (191, 138): 433.0, (191, 139): 1727.0, (191, 140): 339.0, (191, 141): 752.0, (191, 142): 351.0, (191, 143): 361.0, (191, 144): 629.0, (191, 145): 591.0, (191, 146): 673.0, (191, 147): 579.0, (191, 148): 1216.0, (191, 149): 1231.0, (191, 150): 570.0, (191, 151): 2446.0, (191, 152): 536.0, (191, 153): 998.0, (191, 154): 467.0, (191, 155): 578.0, (191, 156): 710.0, (191, 157): 1273.0, (191, 158): 1221.0, (191, 159): 440.0, (191, 160): 335.0, (191, 161): 1387.0, (191, 162): 770.0, (191, 163): 457.0, (191, 164): 1381.0, (191, 165): 268.0, (191, 166): 1093.0, (191, 167): 503.0, (191, 168): 66.0, (191, 169): 293.0, (191, 170): 661.0, (191, 171): 82.0, (191, 172): 326.0, (191, 173): 612.0, (191, 174): 1443.0, (191, 175): 347.0, (191, 176): 594.0, (191, 177): 1639.0, (191, 178): 759.0, (191, 179): 502.0, (191, 180): 775.0, (191, 181): 423.0, (191, 182): 1002.0, (191, 183): 2936.0, (191, 184): 1368.0, (191, 185): 599.0, (191, 186): 457.0, (191, 187): 685.0, (191, 188): 1200.0, (191, 189): 1540.0, (191, 190): 1305.0, (191, 191): 0.0, (191, 192): 603.0, (191, 193): 1358.0, (191, 194): 152.0, (191, 195): 1386.0, (191, 196): 201.0, (191, 197): 1371.0, (191, 198): 770.0, (191, 199): 1026.0, (191, 200): 1012.0}

    # Accounting for circuity factor
    distance = {(w, c): distance[w, c] * 1.17 for w in warehouses for c in customers}

    # Setting the value to be 1 if customer c is within the given high service distance of warehouse w
    high_service_dist_par = {(w, c): 1 if distance[w, c] <= high_service_dist else 0
                             for w in warehouses for c in customers}

    # Setting the value to be 1 if customer c is within the given maximum service distance of warehouse w
    maximum_dist_par = {(w, c): 1 if distance[w, c] <= maximum_dist else 0
                        for w in warehouses for c in customers}
    
    
    return high_service_dist_par, maximum_dist_par, warehouses, customers, customer_demands, distance

# Some Sanity Checks

In [4]:
# In this function, we are testing the input parameters
def test_input(warehouses, customers, customer_demands, distance):
    for c in customers:
        for w in warehouses:
            if distance[w, c] >= 0:
                pass
            else:
                file.write(f'\nDistance between warehouse {w} and customer {c} is not available or invalid')

    for c in customers.keys():
        if customer_demands[c] >= 0:
            pass
        else:
            file.write(f'\nDemand for Customer {c} is not available')

# Optimization Model

## Model 1

In [5]:
# Model_1. This model maximizes the percentage of demand within a distance
def optimal_location_1(high_service_dist_par, maximum_dist_par, warehouses, customers,
                       customer_demands, distance, scenario_name):
    start_time = time.time()
    # problem is initialized
    chens_problem_1 = pulp.LpProblem("chens", pulp.LpMaximize)

    # 'assign_vars' contains the decision variables Yi,j - the assignment of the city to a facility
    assign_vars = pulp.LpVariable.dicts(name="Asssignment",
                                        indexs=[(w, c) for w in warehouses for c in customers],
                                        cat=pulp.LpBinary)
    # 'facility_vars' contains the decision variables Xi - the decision of a facility at city i to be opened or not
    facility_vars = pulp.LpVariable.dicts(name="Open", indexs=[w for w in warehouses], cat=pulp.LpBinary)

    # NEW Decision Variables
    avg_service_dist_decision_var = pulp.LpVariable(name="Avg Service Distance Decision Variable",
                                                    lowBound=0,
                                                    cat=pulp.LpContinuous)
    high_demand_within_dist_decision_var = pulp.LpVariable(name="High Demand within Distance Decision Variable",
                                                           lowBound=0,
                                                           cat=pulp.LpContinuous)
    low_demand_within_dist_decision_var = pulp.LpVariable(name="Low Demand within Distance Decision Variable",
                                                          lowBound=0,
                                                          cat=pulp.LpContinuous)
    # objective function
    demand_within_distance = pulp.lpSum([high_service_dist_par[w, c] * customer_demands[c] * assign_vars[w, c]
                                         for w in warehouses for c in customers])

    """
    If you can't see the args of LpConstraint, here they are:    
    A Pulp LP constraint: LpConstraint(e, sense, name, rhs)
    Parameters:	
    e – an instance of LpExpression
    sense – one of LpConstraintEQ, LpConstraintGE, LpConstraintLE
    name – identifying string
    rhs – numerical value of constraint right hand side
    """

    # NEW Constraints for our new metric variables
    chens_problem_1 += pulp.LpConstraint(
        e=pulp.lpSum([distance[w, c] * customer_demands[c] * assign_vars[w, c]
                      for w in warehouses for c in customers])
          - pulp.lpSum([avg_service_dist_decision_var * customer_demands[c] for c in customers]),
        sense=pulp.LpConstraintEQ,
        name="Metric Var Avg Dist",
        rhs=0)

    chens_problem_1 += pulp.LpConstraint(
        e=pulp.lpSum([high_service_dist_par[w, c] * customer_demands[c] * assign_vars[w, c]
                      for w in warehouses for c in customers])
          - pulp.lpSum([high_demand_within_dist_decision_var * customer_demands[c] / 100
                        for c in customers]),
        sense=pulp.LpConstraintEQ,
        name="Metric Var High Service",
        rhs=0)

    chens_problem_1 += pulp.LpConstraint(e=high_demand_within_dist_decision_var + low_demand_within_dist_decision_var,
                                         sense=pulp.LpConstraintEQ,
                                         name="Metric Var Low Service",
                                         rhs=100)
    
    # Ensures that average service distance is within the given value
    chens_problem_1 += pulp.LpConstraint(
        e=pulp.lpSum([distance[w, c] * customer_demands[c] * assign_vars[w, c]
                      for w in warehouses for c in customers]),
        sense=pulp.LpConstraintLE,
        name="_Avg_Served",
        rhs=avg_service_dist * sum([customer_demands[c] for c in customers]))

    # Ensures that a customer cannot be served from a warehouse which is farther than maximum distance value
    for w in warehouses:
        for c in customers:
            chens_problem_1 += pulp.LpConstraint(e=assign_vars[w, c],
                                                 sense=pulp.LpConstraintLE,
                                                 name=str(w) + "_" + str(c) + "_Max_Served",
                                                 rhs=maximum_dist_par[w, c])

    # Every customer must be served
    for c in customers:
        chens_problem_1 += pulp.LpConstraint(e=pulp.lpSum([assign_vars[w, c] for w in warehouses]),
                                             sense=pulp.LpConstraintEQ,
                                             name=str(c) + "_Served",
                                             rhs=1)

    # Ensure that P and exactly P facilities are opened.  We use the variable name 'number_of_whs' for P
    chens_problem_1 += pulp.LpConstraint(e=pulp.lpSum([facility_vars[w] for w in warehouses]),
                                         sense=pulp.LpConstraintEQ,
                                         name="FacilityCount",
                                         rhs=number_of_whs)

    # An assignment can only exist between facility and customer if the facility is opened. We'll call this the "route"
    for w in warehouses:
        for c in customers:
            chens_problem_1 += pulp.LpConstraint(e=assign_vars[w, c] - facility_vars[w],
                                                 sense=pulp.LpConstraintLE,
                                                 name=str(w) + "_" + str(c) + "_Route",
                                                 rhs=0)

    # Setting problem objective
    chens_problem_1.setObjective(demand_within_distance)
    # The problem data is written to an .lp file
    # chens_problem_1.writeLP("chens_prob_1.lp")

    # The problem is solved using PuLP's choice of Solver
    chens_problem_1.solve()

    file.write('\n\nModel_1')
    print("\nModel_1")
    # The status of the solution is printed to the screen
    # print ("Status:", LpStatus[chens_problem_1.status])
    file.write('\nstatus:' + pulp.LpStatus[chens_problem_1.status])
    print("Optimization Status", pulp.LpStatus[chens_problem_1.status])  # print in Jupyter Notebook
    # print objective
    total_demand = sum(customer_demands.values())
    total_demand_to_warehouse = {w: sum(customer_demands[c] * assign_vars[w, c].varValue for c in customers)
                                 for w in warehouses
                                 if facility_vars[w].varValue > 0.5}
    total_demand_distance = sum(customer_demands[c] * distance[w, c] * assign_vars[w, c].varValue
                                for c in customers for w in warehouses)

    actual_avg_service_dist = total_demand_distance / total_demand

    #     print(total_demand_to_warehouse)
    #     print("Total Demand",total_demand)
    #     print("Total Demand Distance",total_demand_distance)
    #     print("Act Avg",actual_avg_service_dist)
    file.write("\nTotal Demand:" + str(total_demand))
    print("Objective(Demand within Max Service): {:,}".format(pulp.value(chens_problem_1.objective)))
    file.write("\nObjective(Demand within Max Service): " + str(pulp.value(chens_problem_1.objective)))
    print("% of Demand within ", high_service_dist, "km: ",
          round(pulp.value(chens_problem_1.objective) * 100 / total_demand, 2), "%")
    file.write("\n% of Demand within " + str(high_service_dist) + "km: " + str(
        round(pulp.value(chens_problem_1.objective) * 100 / total_demand, 2)) + "%")
    print("Average service distance: {0:.1f} ".format(actual_avg_service_dist), "km")
    file.write("\nAverage service distance: " + str(actual_avg_service_dist) + "km")
    print("New Metric Var Avg Distance ", avg_service_dist_decision_var.varValue)
    print("New Metric High Service within Distance ", high_demand_within_dist_decision_var.varValue)
    print("New Metric Low Service within Distance ", low_demand_within_dist_decision_var.varValue)

    end_time = time.time()
    time_diff = end_time - start_time
    file.write("\nRun Time of model 1 in seconds {:.1f}".format(time_diff))
    print("Run Time of model 1 in seconds {:.1f}".format(time_diff))

    # preparing data to write in excel sheets
    opened_warehouses = []

    for w, val in facility_vars.items():
        if val.varValue > 0.5:
            wh = {
                'Warehouse Key': w,
                'Warehouse City': warehouses[w][1],
                'Country': warehouses[w][2],
                'Lat': warehouses[w][3],
                'Lon': warehouses[w][4],
                'Total Demand to Warehouse': total_demand_to_warehouse[w]
            }
            opened_warehouses.append(wh)

    # converting the list to dataframe
    df_wh = pd.DataFrame.from_records(opened_warehouses)

    df_wh = df_wh[['Warehouse Key', 'Warehouse City', 'Country', 'Total Demand to Warehouse']]
    list_warehouses_open = list(df_wh['Warehouse Key'])

    # writing detailed files
    writer = pd.ExcelWriter(scenario_name + '_detailed.xlsx')
    df_wh.to_excel(writer, 'Opened Warehouses', index=False)

    #     print("Customers Assigned to warehouses")
    customers_assignment = []
    for (w, c), val in assign_vars.items():
        if val.varValue > 0.5:
            cust = {
                'Warehouse': str(warehouses[w][1] + ',' + warehouses[w][2]),
                'Customer': str(customers[c][1] + ',' + customers[c][2]),
                'Customer Demand': customer_demands[c],
                'Distance': distance[w, c],
                'Warehouse Latitude': warehouses[w][3],
                'Warehouse Longitude': warehouses[w][4],
                'Customers Latitude': customers[c][3],
                'Customers Longitude': customers[c][4]
            }
            customers_assignment.append(cust)

    df_cu = pd.DataFrame.from_records(customers_assignment)
    df_cu_copy = df_cu.copy()
    df_cu = df_cu[['Warehouse', 'Customer', 'Distance', 'Customer Demand']]
    df_cu.to_excel(writer, 'Customers Assignment', index=False)

    writer.close()
    return df_cu_copy, list_warehouses_open

## Model 2

In [6]:
# Model_2. This model minimizes the total distance
def optimal_location_2(high_service_dist_par, maximum_dist_par, warehouses, customers,
                       customer_demands, distance, scenario_name):
    start_time = time.time()
    # problem is initialized
    chens_problem_2 = pulp.LpProblem("chens", pulp.LpMinimize)

    # 'assign_vars' contains the decision variables Yi,j - the assignment of the city to a facility
    assign_vars = pulp.LpVariable.dicts(name="Asssignment",
                                        indexs=[(w, c) for w in warehouses for c in customers],
                                        cat=pulp.LpBinary)
    # 'facility_vars' contains the decision variables Xi - the decision of a facility at city i to be opened or not
    facility_vars = pulp.LpVariable.dicts(name="Open", indexs=[w for w in warehouses], cat=pulp.LpBinary)

    # objective function
    total_distance = pulp.lpSum([distance[w, c] * customer_demands[c] * assign_vars[w, c]
                                 for w in warehouses for c in customers])

    """
    If you can't see the args of LpConstraint, here they are:    
    A Pulp LP constraint: LpConstraint(e, sense, name, rhs)
    Parameters:	
    e – an instance of LpExpression
    sense – one of LpConstraintEQ, LpConstraintGE, LpConstraintLE
    name – identifying string
    rhs – numerical value of constraint right hand side
    """

    # Ensures that a certain demand is within the service distance
    chens_problem_2 += pulp.LpConstraint(
        e=pulp.lpSum([high_service_dist_par[w, c] * customer_demands[c] * assign_vars[w, c]
                      for w in warehouses for c in customers]),
        sense=pulp.LpConstraintGE,
        name="_Avg_Served",
        rhs=high_service_demand)

    # Ensures that a customer cannot be served from a warehouse which is farther than maximum distance value
    for w in warehouses:
        for c in customers:
            chens_problem_2 += pulp.LpConstraint(e=assign_vars[w, c],
                                                 sense=pulp.LpConstraintLE,
                                                 name=str(w) + "_" + str(c) + "_Max_Served",
                                                 rhs=maximum_dist_par[w, c])

    # Every customer must be served
    for c in customers:
        chens_problem_2 += pulp.LpConstraint(e=pulp.lpSum([assign_vars[w, c] for w in warehouses]),
                                             sense=pulp.LpConstraintEQ,
                                             name=str(c) + "_Served",
                                             rhs=1)

    # Ensure that P and exactly P facilities are opened.  We use the variable name 'number_of_whs' for P
    chens_problem_2 += pulp.LpConstraint(e=pulp.lpSum([facility_vars[w] for w in warehouses]),
                                         sense=pulp.LpConstraintEQ,
                                         name="FacilityCount",
                                         rhs=number_of_whs)

    # An assignment can only exist between facility and customer if the facility is opened. We'll call this the "route"
    for w in warehouses:
        for c in customers:
            chens_problem_2 += pulp.LpConstraint(e=assign_vars[w, c] - facility_vars[w],
                                                 sense=pulp.LpConstraintLE,
                                                 name=str(w) + "_" + str(c) + "_Route",
                                                 rhs=0)

    # Setting problem objective
    chens_problem_2.setObjective(total_distance)
    # The problem data is written to an .lp file
    # chens_problem_2.writeLP("chens_prob_2.lp")

    # The problem is solved using PuLP's choice of Solver
    chens_problem_2.solve()

    file.write('\n\nModel_2')
    print("\nModel_2")
    # The status of the solution is printed to the screen
    # print ("Status:", LpStatus[chens_problem_2.status])
    file.write('\nstatus:' + pulp.LpStatus[chens_problem_2.status])
    print("Optimization Status", pulp.LpStatus[chens_problem_2.status])  # print in Jupyter Notebook
    # print objective
    total_demand = sum(customer_demands.values())
    total_demand_to_warehouse = {w: sum(customer_demands[c] * assign_vars[w, c].varValue for c in customers)
                                 for w in warehouses
                                 if facility_vars[w].varValue > 0.5}

    # print (total_demand_to_warehouse)
    # print ("Total Demand",total_demand)
    file.write("\nTotal Demand:" + str(total_demand))
    print("Objective(Total Demand-Distance): ", pulp.value(chens_problem_2.objective))
    file.write("\nObjective: " + str(pulp.value(chens_problem_2.objective)))
    print("% of Demand within ", high_service_dist, "km: ", round(high_service_demand * 100 / total_demand, 2), "%")
    file.write("\n% of Demand within " + str(high_service_dist) + "km: " + str(
        round(high_service_demand * 100 / total_demand, 2)) + "%")
    print("Equivalent Objective(Average service distance): ",
          round(pulp.value(chens_problem_2.objective) / total_demand, 1), "km")
    file.write(
        "\nAverage service distance: " + str(round(pulp.value(chens_problem_2.objective) / total_demand, 1)) + "km")

    end_time = time.time()
    time_diff = end_time - start_time
    file.write("\nRun Time of model 2 in seconds {:.1f}".format(time_diff))
    print("Run Time of model 2 in seconds {:.1f}".format(time_diff))

    # preparing data to write in excel sheets
    opened_warehouses = []

    for w, val in facility_vars.items():
        if val.varValue > 0.5:
            wh = {
                'Warehouse Key': w,
                'Warehouse City': warehouses[w][1],
                'Country': warehouses[w][2],
                'Lat': warehouses[w][3],
                'Lon': warehouses[w][4],
                'Total Demand to Warehouse': total_demand_to_warehouse[w]
            }
            opened_warehouses.append(wh)

    # converting the list to dataframe
    df_wh = pd.DataFrame.from_records(opened_warehouses)

    df_wh = df_wh[['Warehouse Key', 'Warehouse City', 'Country', 'Total Demand to Warehouse']]
    list_warehouses_open = list(df_wh['Warehouse Key'])

    # writing detailed files
    writer = pd.ExcelWriter(scenario_name + '_detailed.xlsx')
    df_wh.to_excel(writer, 'Opened Warehouses', index=False)

    #     print("Customers Assigned to warehouses")
    customers_assignment = []
    for (w, c), val in assign_vars.items():
        if val.varValue > 0.5:
            cust = {
                'Warehouse': str(warehouses[w][1] + ',' + warehouses[w][2]),
                'Customer': str(customers[c][1] + ',' + customers[c][2]),
                'Customer Demand': customer_demands[c],
                'Distance': distance[w, c],
                'Warehouse Latitude': warehouses[w][3],
                'Warehouse Longitude': warehouses[w][4],
                'Customers Latitude': customers[c][3],
                'Customers Longitude': customers[c][4]
            }
            customers_assignment.append(cust)

    df_cu = pd.DataFrame.from_records(customers_assignment)
    df_cu_copy = df_cu.copy()
    df_cu = df_cu[['Warehouse', 'Customer', 'Distance', 'Customer Demand']]
    df_cu.to_excel(writer, 'Customers Assignment', index=False)

    writer.close()
    return df_cu_copy, list_warehouses_open

# Visulization

## Input data

In [7]:
# In this function, we are visualizing the input data-- the data before the optimization has run
def input_visual(warehouses, customers):
    warehouse_list = []
    for w, values in warehouses.items():
        wh = {
            'text': 'Warehouse-' + values[1],
            'Country': values[2],
            'lat': values[3],
            'long': values[4],
            'cnt': 10000000,
            'size': 30,
            'color': 'rgba(0, 100, 0)'
        }
        warehouse_list.append(wh)

    customer_list = []
    for c, values in customers.items():
        cust = {
            'text': 'Customer-' + values[1],
            'Country': values[2],
            'lat': values[3],
            'long': values[4],
            'cnt': customer_demands[c],
            'size': 3,
            'color': 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)

    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] = "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] = "circle"
    df = df.append(df_cust, ignore_index=True)

    locations = [dict(
        type='scattergeo',
        locationmode='country names',
        lon=df['long'],
        lat=df['lat'],
        hoverinfo='text',
        text=df['text'],
        mode='markers',
        marker=dict(
            size=df['size'],
            color=df['color'],
            symbol=df['shape'],
            line=dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            ),
        ))]

    layout = dict(
        title="Chen's Cosmetics - Input",
        showlegend=False,
        geo=dict(
            scope='asia',
            projection=dict(type='mercator'),
            showland=True,
            landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',
        ),
    )
    py.offline.plot({"data": locations, "layout": layout}, filename=scenario_name + '_input.html')

## Output data

In [8]:
# In this function, we are visualizing the output data-- the data after the optimization has run
def output_visual(warehouses, customers, df_cu, wh_loc, scenario_name, model_name):
    warehouse_list = []
    for w in wh_loc:
        wh = {
            'text': 'Warehouse-' + warehouses[w][1],
            'Country': warehouses[w][2],
            'lat': warehouses[w][3],
            'long': warehouses[w][4],
            'cnt': 10000000,
            'size': 30,
            'color': 'rgba(0, 100, 0)'
        }
        warehouse_list.append(wh)

    customer_list = []
    for c, values in customers.items():
        cust = {
            'text': 'Customer-' + values[1],
            'Country': values[2],
            'lat': values[3],
            'long': values[4],
            'cnt': customer_demands[c],
            'size': 3,
            'color': 'rgb(255, 0, 0)'
        }
        customer_list.append(cust)

    df = pd.DataFrame.from_records(warehouse_list)
    df['shape'] = "triangle-down"
    df_cust = pd.DataFrame.from_records(customer_list)
    df_cust['shape'] = "circle"
    df = df.append(df_cust, ignore_index=True)

    paths = []
    for i in range(len(df_cu)):
        paths.append(
            dict(
                type='scattergeo',
                locationmode='country names',
                lon=[df_cu['Warehouse Longitude'][i], df_cu['Customers Longitude'][i]],
                lat=[df_cu['Warehouse Latitude'][i], df_cu['Customers Latitude'][i]],
                mode='lines',
                line=dict(
                    width=1,
                    color='red',
                ),
                opacity=0.8,
            )
        )

    locations = [dict(
        type='scattergeo',
        locationmode='country names',
        lon=df['long'],
        lat=df['lat'],
        hoverinfo='text',
        text=df['text'],
        mode='markers',
        marker=dict(
            size=df['size'],
            color=df['color'],
            symbol=df['shape'],
            line=dict(
                width=3,
                color='rgba(68, 68, 68, 0)'
            ),
        ))]

    layout = dict(
        title="Chen's Cosmetics - Output: " + model_name,
        showlegend=False,
        geo=dict(
            scope='asia',
            projection=dict(type='mercator'),
            showland=True,
            landcolor='rgb(243, 243, 243)',
            countrycolor='rgb(204, 204, 204)',
        ),
    )

    py.offline.plot({"data": locations + paths, "layout": layout}, filename=scenario_name + '_output.html')

# Run Block

In [9]:
# getting input data
high_service_dist_par, maximum_dist_par, warehouses, customers, customer_demands, distance = get_data()

# print("loaded input data")
# distance_band = [distance_band_1, distance_band_2, distance_band_3, distance_band_4]

# opening text file in write mode
file = open(scenario_name + 'summary' + '.txt', "w")
file.write('loaded input data')
print("loaded input data")  # Also print to Jupyter

# testing input data
file.write('\nTesting Input Data')
test_input(warehouses, customers, customer_demands, distance)

file.write('\nBuilding the models')

df_assign_vars_1, list_facility_vars_1 = optimal_location_1(high_service_dist_par, maximum_dist_par, warehouses, 
                                                            customers, customer_demands, distance, scenario_name)

df_assign_vars_2, list_facility_vars_2 = optimal_location_2(high_service_dist_par, maximum_dist_par, warehouses,
                                                            customers, customer_demands, distance, scenario_name)

# printing the maps
if input_map:
    input_visual(warehouses, customers)
if output_map:
    output_visual(warehouses, customers, df_assign_vars_1, list_facility_vars_1, scenario_name, 'Model 1')
    # output_visual(warehouses, customers, df_assign_vars_2, list_facility_vars_2, scenario_name, 'Model 2')

# closing the file
file.close()

loaded input data


<ipython-input-5-7bdc6525e6d4>:9: DeprecationWarning: 'indexs' is deprecated; use 'indices'.
  assign_vars = pulp.LpVariable.dicts(name="Asssignment",
<ipython-input-5-7bdc6525e6d4>:13: DeprecationWarning: 'indexs' is deprecated; use 'indices'.
  facility_vars = pulp.LpVariable.dicts(name="Open", indexs=[w for w in warehouses], cat=pulp.LpBinary)



Model_1
Optimization Status Optimal
Objective(Demand within Max Service): 131,645,389.0
% of Demand within  600 km:  66.06 %
Average service distance: 660.5  km
New Metric Var Avg Distance  660.47494
New Metric High Service within Distance  66.063867
New Metric Low Service within Distance  33.936133
Run Time of model 1 in seconds 0.5

Model_2
Optimization Status Optimal
Objective(Total Demand-Distance):  116090197990.64998
% of Demand within  600 km:  6.61 %
Equivalent Objective(Average service distance):  582.6 km
Run Time of model 2 in seconds 0.4
